## CPEX Field Campaign Series ERA5 Reanalysis of Convective Inflow and Convergence

This script is an adapted version of "CPEXCV_ERA5_3DWind_EXAMPLE_Giselle.ipynb"

Code for plotting streamlines and convergence from ERA5 Reanalysis (https://www.ecmwf.int/en/forecasts/dataset/ecmwf-reanalysis-v5) at select times and pressure levels.

(Note from Giselle: the smoothing on the example plot isn’t needed. There’s some pressure levels for which the smoothed streamlines are not correctly plotted, and I so removed the smoothing code on the template, so you don’t have to worry about that. If you want to use the example code, I would suggest to remove the smoothing for the streamlines.)

This notebook plots ECMWF ERA5 reanalysis data (3-D hourly gridded winds with 0.25 degree resolution) to give context for the large-scale flow and where the near-storm dropsondes are located relative to the convective inflow region(s).

#### <span style="color:purple"> Plotting streamlines at each hour: </span>
 
- Low-level Inflow: $975 \: hPa, 950 \: hPa, 925 \: hPa$
- Mid-level Inflow: $775 \: hPa, 750 \: hPa, 700 \: hPa$

- Low-level Convergence: $1000 \: hPa, 975 \: hPa, 950 \: hPa, 925 \: hPa$

#### Resources

[ECMWF Reanalysis ERA5](https://www.ecmwf.int/en/forecasts/dataset/ecmwf-reanalysis-v5) <br>

### ERA5 Reanalysis -- 3D Winds

In [ ]:
import os
import sys
import math
import h5py
import xarray as xr
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm  #to get python's normal library of colormaps
import matplotlib.colors as mplc

import cartopy.crs as ccrs
import cartopy.feature as cfeature
#from cartopy.util import add_cyclic_point
#from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from datetime import datetime
from datetime import timedelta

import metpy.calc as mpcalc
import metpy.plots as mplots

from PIL import Image
import icartt            #needed to read .ict files


In [ ]:
###the only variables you need to change are the 2 in this cell
pressures_to_plot_stream = [975, 950, 925, 775, 750, 700]   #desired pressure levels to be plotted for streamlines
pressures_to_plot_conv = [1000, 975, 950, 925]              #desired pressure levels to be plotted for convergence
pressures_to_plot_RH = [850, 800, 750, 700]                 #desired pressure levels to be plotted for RH for convergence plots

#dict of desired flight dates (key) to plot ERA5 streamlines/convergence for and
#their associated list of desired UTC hours to be plotted (e.g., convective case hours/time range)

#CPEX-CV convective cases (also plotting +/- 4 hours around convective cases)
case_dict_stream = {'20220906': [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
                   '20220907': [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
                   '20220909': [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
                   '20220910': [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
                   '20220914': [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
                   '20220916': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
                   '20220920': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
                   '20220922': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
                   '20220923': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
                   '20220926': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
                   '20220929': [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
                   '20220930': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]}

case_dict_conv = case_dict_stream

# #CPEX and CPEX-AW convective cases
# case_dict_stream = {'20170601': [18, 19, 20, 21, 22],      
#              '20170602': [18, 19, 20, 21, 22],
#              '20170606': [19, 20, 21, 22],
#              '20170610': [20, 21, 22, 23],
#              '20170611': [17, 18, 19, 20, 21, 22],
#              '20170615': [19, 20, 21],
#              '20170616': [19, 20, 21, 22],
#              '20170617': [19, 20, 21, 22],
#              '20170619': [17, 18, 19, 20, 21, 22, 23],
#              '20170620': [17, 18, 19, 20, 21, 22],
#              '20170624': [18, 19, 20, 21, 22],
#              '20210821': [22, 23],
#              '20210822': [0, 1, 2],
#              '20210824': [18, 19, 20, 21]}

# #CPEX(-AW) cases 13 and 16
# case_dict_stream = {'20170611': [14, 15, 16, 17, 18, 19, 20, 21, 22],
#                   '20210824': [15, 16, 17, 18, 19, 20, 21]}
# case_dict_conv = {'20170611': [14, 15, 16, 17, 18, 19, 20, 21, 22],
#                   '20210824': [15, 16, 17, 18, 19, 20, 21]}

# #for testing
# case_dict_stream = {'20170611': [19]}   #for testing
# case_dict_conv   = {'20210824': [15]}   #for testing


In [ ]:
# file_date = '20220906'
# hr = 19

# ###get locations of the dropsonde/Navigation/ERA5 folder and read the appropriate files in
# day_folder = os.path.join(os.getcwd(), file_date)

# #dropsonde data
# drop_csv_path = os.path.join(day_folder, 'final_dropsonde_' + file_date + '.csv')
# drop_csv = pd.read_csv(drop_csv_path)

# if file_date[:4] == '2017':
#     campaign = 'CPEX'
#     drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
# elif file_date[:4] == '2021':
#     campaign = 'CPEXAW'
#     drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
# elif file_date[:4] == '2022':
#     campaign = 'CPEXCV'
#     drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')
# else:
#     pass

# #ERA5 data
# era5_folder = os.path.join(os.getcwd(), 'ERA5_Reanalysis_Data')
# era5_path = os.path.join(era5_folder, campaign + '_ERA5_Reanalysis_Hourly_Precip_Avg.nc')
# ds_precip = xr.open_dataset(era5_path)
# ds_precip

# # era5_path = os.path.join(era5_folder, campaign + '_ERA5_Reanalysis_Hourly_Precip_Instant.nc')
# # ds_precip2 = xr.open_dataset(era5_path)
# # ds_precip2


#### Time-mean total precipitation rate with flight track and dropsondes overlaid for each desired flight and time ranges in case_dict_stream

In [ ]:
#set some baseline plot displays

#matplotlib.rcParams['axes.facecolor'] = [0.9,0.9,0.9]
matplotlib.rcParams['axes.labelsize'] = 20
matplotlib.rcParams['axes.titlesize'] = 20
matplotlib.rcParams['axes.labelweight'] = 'bold'
matplotlib.rcParams['axes.titleweight'] = 'bold'
matplotlib.rcParams['xtick.labelsize'] = 16
matplotlib.rcParams['ytick.labelsize'] = 16
matplotlib.rcParams['legend.fontsize'] = 16
#matplotlib.rcParams['legend.facecolor'] = 'w'
#matplotlib.rcParams['axes.facecolor'] = 'w'
matplotlib.rcParams['font.family'] = 'arial'
matplotlib.rcParams['hatch.linewidth'] = 0.3

#file_date is the date on which the desired flight took place
#utc_hours_to_plot is the desired UTC hours to be plotted for the given flight (e.g., convective case hours/time range)
for file_date, utc_hours_to_plot in case_dict_stream.items():

    print (file_date + ' streamline plots in progress...')
    
    ###get locations of the dropsonde/Navigation/ERA5 folder and read the appropriate files in
    day_folder = os.path.join(os.getcwd(), file_date)

    #dropsonde data
    drop_csv_path = os.path.join(day_folder, 'final_dropsonde_' + file_date + '.csv')
    drop_csv = pd.read_csv(drop_csv_path)

    if file_date[:4] == '2017':
        campaign = 'CPEX'
        drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
    elif file_date[:4] == '2021':
        campaign = 'CPEXAW'
        drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
    elif file_date[:4] == '2022':
        campaign = 'CPEXCV'
        drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')
    else:
        pass

    #ERA5 data
    era5_folder = os.path.join(os.getcwd(), 'ERA5_Reanalysis_Data')
    era5_path = os.path.join(era5_folder, campaign + '_ERA5_Reanalysis_Hourly_Pressure.nc')
    ds = xr.open_dataset(era5_path)

    era5_path_precip = os.path.join(era5_folder, campaign + '_ERA5_Reanalysis_Hourly_Precip_Avg.nc')
    ds_precip = xr.open_dataset(era5_path_precip)

    #Navigation data
    nav_folder = os.path.join(day_folder, 'Nav_files')

    for x in os.listdir(nav_folder):
        if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
            os.remove(os.path.join(nav_folder, x))

    nav_ict_path = os.path.join(nav_folder, os.listdir(nav_folder)[0])  #only one nav file per flight

    if campaign != 'CPEX':  #campaign either CPEXAW or CPEXCV

        nav_ict = icartt.Dataset(nav_ict_path)    #open the ict file with icartt
        flight_lat = nav_ict.data["Latitude"]     #nav latitude, just a normal 1-D array
        flight_lon = nav_ict.data["Longitude"]    #nav longitude, just a normal 1-D array

    else:  #for CPEX navigation files, open the CSV file with pandas
           #(Navigation files for CPEX (2017) are originally .kmz not .ict,
           #so I converted them to CSV for free using https://www.gpsvisualizer.com/convert_input

        nav_ict = pd.read_csv(nav_ict_path)       #open the ict file with pandas instead
        flight_lat = nav_ict["latitude"].values   #nav latitude, just a normal 1-D array
        flight_lon = nav_ict["longitude"].values  #nav longitude, just a normal 1-D array    

    #flight track lat/lon extent [West,East,South,North] for plotting, giving an XX degree buffer around the flight track   
    campaign_extent = [flight_lon.min() - 2.5, flight_lon.max() + 2.5, flight_lat.min() - 2.5, flight_lat.max() + 2.5]


    ###calculate each near-storm dropsonde's mean lat/lon and add the sonde's time and mean lat/lon to a list to be plotted

    drop_coords_and_time = []   #format: longitude, latitude, time (HHSS)

    df_drop = pd.read_csv(drop_metric_filepath)
    df_drop_use = df_drop[df_drop['Date'] == int(file_date)].copy()

    for x in range(len(df_drop_use)):
        date = str(df_drop_use['Date'].iloc[x])
        time = str(df_drop_use['Time'].iloc[x]).zfill(6)
        sonde_datetime = date[:4] + '-' + date[4:6] + '-' + date[6:] + ' ' + time[:2] + ':' + time[2:4] + ':' + time[4:]

        drop_csv_use = drop_csv[drop_csv['Time [UTC]'] == sonde_datetime].copy()
        drop_mean_lon = drop_csv_use['Longitude [deg]'].mean()
        drop_mean_lat = drop_csv_use['Latitude [deg]'].mean()

        sonde_info = [drop_mean_lon, drop_mean_lat, time[:4]]
        drop_coords_and_time.append(sonde_info)

        
    ###create an XX-panel plot of streamlines at XX-different pressure levels 
        ###for each desired hour of the given day

    half_pres_levs = math.ceil(len(pressures_to_plot_stream) / 2)   #to determine size of figure and number of subplots to generate (rounded up to the nearest multiple of 2)

    for hr in utc_hours_to_plot:
        
        hr2 = str(hr).zfill(2)
        hour_prior = str(hr - 1).zfill(2)

        #MIMIC TPW data
           ##https://bin.ssec.wisc.edu/pub/mtpw2/data/
        tpw_folder = os.path.join(day_folder, 'MIMIC_TPW_files')
        tpw_path = os.path.join(tpw_folder, 'comp' + file_date + '.' + hr2 + '0000.nc')
        ds_tpw = xr.open_dataset(tpw_path)

        #GPM IMERG data (see IMERG.ipynb for more how to more generally download and plot IMERG data)
           ##https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Read%20IMERG%20Data%20Using%20Python
           ##https://disc.gsfc.nasa.gov/datasets?keywords=imerg&page=1
           #0.1 x 0.1 gridded data, half-hourly means, using the half hour BEFORE the desired hour
        imerg_folder = os.path.join(day_folder, 'IMERG_files')
        
        for x in os.listdir(imerg_folder):
            if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
                os.remove(os.path.join(imerg_folder, x))
                
            #minutes and seconds automatically revert to zero (hour = 0, seconds = 0) for '%Y%m%d%H'
            elif (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(seconds = 1), '%H%M%S') in x) and (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(minutes = 30), '%H%M%S') in x):
                imerg_file = x
                break
            else:
                imerg_file = 'Could not find the desired IMERG file'
        
        #confirm that the IMERG file is from the correct day (if hr2 == '00', then this will be the previous day)
        assert (file_date in imerg_file) or (hr2 == '00'), 'IMERG file not from the correct day'
        
        imerg_path = os.path.join(imerg_folder, imerg_file)
        ds_imerg = h5py.File(imerg_path, 'r')
        
        imerg_lons = ds_imerg['Grid/lon'][:]   #Longitude Shape: (3600,)
        imerg_lats = ds_imerg['Grid/lat'][:]   #Latitude Shape: (1800,)
        imerg_lons, imerg_lats = np.meshgrid(imerg_lons, imerg_lats)  #Long and lat grid shape: (1800, 3600) 
        
        imerg_precip = ds_imerg['Grid/precipitation'][0][:][:]  #Original Precip Shape: (1, 3600, 1800) = (time, lon, lat)
        imerg_precip = np.transpose(imerg_precip)               #New Precip Shape after transpose: (1800, 3600)
        
        #mask blank data
        imerg_precip_masked = np.ma.masked_where(imerg_precip < 0, imerg_precip)  #masks blank and bad data first (if blank data is -999 instead of NaN)
        imerg_precip_masked = np.ma.masked_where(np.isnan(imerg_precip_masked), imerg_precip_masked)  #masks NaN values (not masked in previous line)        
        
        data_proj = ccrs.PlateCarree()

        group_fig = plt.figure(figsize = (12 * half_pres_levs, 5 * half_pres_levs))   #initialize the streamline figure for the given hour

        precip_rate = ds_precip.avg_tprate.sel(valid_time = file_date)                                 #Time-mean total precipitation rate (kg/m2/s)
        precip_rate = precip_rate.sel(valid_time = precip_rate.valid_time.dt.hour.isin(hr)) * 3600     #Time-mean total precipitation rate (mm/hr)

        for ii, pres_lev in enumerate(pressures_to_plot_stream):
            ax = group_fig.add_subplot(2, half_pres_levs, ii+1, projection = data_proj)        

            uwnd = ds.u.sel(time = file_date).sel(level = pres_lev)  #zonal winds (m/s)
            uwnd = uwnd.sel(time = uwnd.time.dt.hour.isin(hr))       #zonal winds (m/s)

            vwnd = ds.v.sel(time = file_date).sel(level = pres_lev)  #meridional winds (m/s)
            vwnd = vwnd.sel(time = vwnd.time.dt.hour.isin(hr))       #meridional winds (m/s)
            
            rh = ds.r.sel(time = file_date).sel(level = pres_lev)    #relative humidity (%)
            rh = rh.sel(time = rh.time.dt.hour.isin(hr))             #relative humidity (%)

            ##Smoothing (source: Hannah Zanowski) --> not recommended, see top of document
                ##Metpy smooth_n_point (data to be smoothed, number of points to use in smoothing (5 to 9 are valid), and number of times the smoother is applied)
                    ##see https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.smooth_n_point.html for more info
            #uwnd_smoothed = mpcalc.smooth_n_point(uwnd,9,10)
            #vwnd_smoothed = mpcalc.smooth_n_point(vwnd,9,10)

            ax.set_title('MIMIC TPW, GPM IMERG, and \nERA5 Precip. Rate (%s, %s UTC)' % (file_date, hr2))
            #ax.set_title('ERA5 %i hPa RH, GPM IMERG, and \nERA5 Precip. Rate (%s, %s UTC)' % (pres_lev, file_date, hr2))
            ax.set_extent(campaign_extent, ccrs.PlateCarree()) #lat/lon bounds are [West,East,South,North]

            # Add land, coastlines, and borders
            #ax.add_feature(cfeature.LAND, facecolor='0.8')
            ax.coastlines(ls = '-', linewidth = 0.5, color = 'gray')
            
            #plot MIMIC TPW
            tpw_levels = np.arange(0, 70.5, 2)
            pm0 = ax.contourf(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, levels = tpw_levels,
                              extend = 'max', cmap = cm.jet, transform = data_proj)
#             pm0 = ax.pcolormesh(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, vmin = 0, vmax = 70,
#                                 cmap = cm.jet, transform = data_proj, zorder = 0)

        #     #plot ERA5 RH
        #     tpw_levels = np.arange(0, 100.1, 2)  #actually RH levels, but keeping the tpw_levels name because we use it elsewhere
        #     pm0 = ax.contourf(ds.longitude, ds.latitude, rh[0].values, levels = tpw_levels,
        #                       extend = 'max', cmap = cm.jet, transform = data_proj, alpha = 0.6)
        # #             pm0 = ax.pcolormesh(ds.longitude, ds.latitude, rh[0].values, vmin = 0, vmax = 70,
        # #                                 cmap = cm.jet, transform = data_proj, zorder = 0)
    
            #plot IMERG Rain Rate
            pm1 = ax.contourf(imerg_lons, imerg_lats, imerg_precip_masked, 
                              levels = np.logspace(np.log10(0.1), np.log10(40), num = len(tpw_levels)), 
                              norm = 'log', extend = 'max', 
                              cmap = cm.jet, transform = data_proj, zorder = 1)
#             pm1 = ax.pcolormesh(imerg_lons, imerg_lats, imerg_precip_masked, 
#                                 norm = mplc.LogNorm(vmin = 0.1, vmax = 40), 
#                                 cmap = cm.jet, transform = data_proj, zorder = 1) 

            #Gridlines
            gl = ax.gridlines(crs = ccrs.PlateCarree(), draw_labels = True, 
                              linewidth = 0.5, color = 'gray', alpha = 0.5, linestyle = '--', zorder = 2)
            gl.top_labels = False
            gl.right_labels = False

            # #plot ERA5 streamlines
            # ax.streamplot(ds.longitude, ds.latitude, uwnd[0].values, vwnd[0].values,
            #               color = 'k', linewidth = 0.6, density = 1.0, transform = data_proj, zorder = 3)

            #plot ERA5 time-mean total precipitation rate (mm/hr)
            ax.contour(ds_precip.longitude, ds_precip.latitude, precip_rate[0].values, levels = [0.1, 1, 5, 10, 20, 40],
                       colors = 'k', linewidths = 1.5, transform = data_proj, zorder = 3)

            #plot flight track
            ax.plot(flight_lon, flight_lat, color = 'white', linewidth = 1.5, zorder = 4)            
            
            #plot near-storm dropsonde locations for the given flight 
                #if the dropsonde was deployed within 30 minutes (1-hr total range) of the given hour
                    #NOTE: Dropsondes with no wind data don't have GPS data either (5 of them total)
            for sonde in drop_coords_and_time:
                if (sonde[2][:2] == hour_prior and int(sonde[2][2:4]) >= 30) or (sonde[2][:2] == hr2 and int(sonde[2][2:4]) < 30):
                    #ax.scatter(sonde[0], sonde[1], marker = f'${sonde[2]}$', color = 'b', s = 300)
                    ax.scatter(sonde[0], sonde[1], marker = '*', color = 'k', zorder = 5, s = 300)

            #same as above, but labeling the dropsondes by the order that they appear in the
                #Dropsonde_Metric_Calculations.csv, NOT IN CHRONOLOGICAL ORDER!!!
            #for z, sonde in enumerate(drop_coords_and_time):
                #ax.scatter(sonde[0], sonde[1], marker = f'${z + 1}$', color = 'b', s = 120, zorder = 5) 
                
            #plotting the colorbars
            #cbar0 = group_fig.colorbar(pm0, ax = ax, orientation = 'vertical', shrink = 0.75, pad = 0.25)
            #cbar0.set_label('TPW [mm]')
            #cbar0.ax.yaxis.set_ticks_position('left')
            #cbar0.ax.yaxis.set_label_position('left')
            
            #this works with GeoAxes (i.e., Cartopy's map projections)
            if ii == len(pressures_to_plot_stream) - 1:
                
                #MIMIC TPW colorbar
                ticks_tpw = np.arange(0, 70.5, 10, dtype = int)
                #cax = group_fig.add_axes([ax.get_position().x1+0.05, ax.get_position().y0, 0.02, ax.get_position().height])
                cax0 = group_fig.add_axes([ax.get_position().x1 + 0.05, ax.get_position().y0, 0.02, 0.75])
                cbar0 = group_fig.colorbar(pm0, cax = cax0, ticks = ticks_tpw)
                cbar0.set_label('TPW [mm]')
                cbar0.ax.set_yticklabels(list(map(str, list(ticks_tpw))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
                cbar0.ax.yaxis.set_ticks_position('left')
                cbar0.ax.yaxis.set_label_position('left')

                # #ERA5 RH colorbar
                # ticks_rh = np.arange(0, 100.5, 10, dtype = int)
                # #cax = group_fig.add_axes([ax.get_position().x1+0.05, ax.get_position().y0, 0.02, ax.get_position().height])
                # cax0 = group_fig.add_axes([ax.get_position().x1 + 0.05, ax.get_position().y0, 0.02, 0.75])
                # cbar0 = group_fig.colorbar(pm0, cax = cax0, ticks = ticks_rh)
                # cbar0.set_label('Relative Humidity [%]')
                # cbar0.ax.set_yticklabels(list(map(str, list(ticks_rh))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
                # cbar0.ax.yaxis.set_ticks_position('left')
                # cbar0.ax.yaxis.set_label_position('left')
                
                #IMERG colorbar
                ticks_imerg = np.array([0.1, 1, 5, 10, 20, 40], dtype = float)
                cax1 = group_fig.add_axes([ax.get_position().x1 + 0.05, ax.get_position().y0, 0.02, 0.75])
                cbar1 = group_fig.colorbar(pm1, cax = cax1, ticks = ticks_imerg)
                cbar1.set_label('IMERG [mm hr$\\bf{^{-1}}$]')
                cbar1.ax.set_yticklabels(list(map(str, list(ticks_imerg))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
                cbar1.ax.yaxis.set_ticks_position('right')
                cbar1.ax.yaxis.set_label_position('right')
                
        ds_tpw.close()
        ds_imerg.close()                
                
        #plt.tight_layout()
        #plt.subplots_adjust(wspace = 0.1)

        #save the figure
        #plot_save_name = f'ERA5_streamlines_RH_{hr2}UTC.png'
        #plot_save_name = f'ERA5_streamlines_TPW_{hr2}UTC.png'
        plot_save_name = f'ERA5_precip_TPW_{hr2}UTC.png'
        plt.savefig(os.path.join(day_folder, plot_save_name), bbox_inches = 'tight')
        #plt.show()  #plt.show() must come after plt.savefig() in order for the image to save properly
        #plt.clf()   #supposedly speeds things up? According to: https://www.youtube.com/watch?v=jGVIZbi9uMY
        plt.close()
        plt.clf()    #if placing this after plt.close(), may release memory related to the figure (https://stackoverflow.com/questions/741877/how-do-i-tell-matplotlib-that-i-am-done-with-a-plot)

        ##decrease file size of the image by 66% without noticeable image effects (if using Matplotlib)
        ##(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
        im = Image.open(os.path.join(day_folder, plot_save_name))

        try:
            im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
        except:
            #use this for older version of PIL/Pillow if the above line doesn't work, 
            #though this line will have isolated, extremely minor image effects due to 
            #only using 256 colors instead of the 3-element RGB scale
            im2 = im.convert('P')

        im2.save(os.path.join(day_folder, plot_save_name))
        im.close()
        im2.close()

    ds.close()
    ds_precip.close()
    
    print (file_date + ' precip. rate plots complete!\n')


#### Streamlines at Various Pressure Levels with flight track and dropsondes overlaid for each desired flight and time ranges in case_dict_stream

In [ ]:
# #set some baseline plot displays

# #matplotlib.rcParams['axes.facecolor'] = [0.9,0.9,0.9]
# matplotlib.rcParams['axes.labelsize'] = 20
# matplotlib.rcParams['axes.titlesize'] = 20
# matplotlib.rcParams['axes.labelweight'] = 'bold'
# matplotlib.rcParams['axes.titleweight'] = 'bold'
# matplotlib.rcParams['xtick.labelsize'] = 16
# matplotlib.rcParams['ytick.labelsize'] = 16
# matplotlib.rcParams['legend.fontsize'] = 16
# #matplotlib.rcParams['legend.facecolor'] = 'w'
# #matplotlib.rcParams['axes.facecolor'] = 'w'
# matplotlib.rcParams['font.family'] = 'arial'
# matplotlib.rcParams['hatch.linewidth'] = 0.3

# #file_date is the date on which the desired flight took place
# #utc_hours_to_plot is the desired UTC hours to be plotted for the given flight (e.g., convective case hours/time range)
# for file_date, utc_hours_to_plot in case_dict_stream.items():

#     print (file_date + ' streamline plots in progress...')
    
#     ###get locations of the dropsonde/Navigation/ERA5 folder and read the appropriate files in
#     day_folder = os.path.join(os.getcwd(), file_date)

#     #dropsonde data
#     drop_csv_path = os.path.join(day_folder, 'final_dropsonde_' + file_date + '.csv')
#     drop_csv = pd.read_csv(drop_csv_path)

#     if file_date[:4] == '2017':
#         campaign = 'CPEX'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
#     elif file_date[:4] == '2021':
#         campaign = 'CPEXAW'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
#     elif file_date[:4] == '2022':
#         campaign = 'CPEXCV'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')
#     else:
#         pass

#     #ERA5 data
#     era5_folder = os.path.join(os.getcwd(), 'ERA5_Reanalysis_Data')
#     era5_path = os.path.join(era5_folder, campaign + '_ERA5_Reanalysis_Hourly_Pressure.nc')
#     ds = xr.open_dataset(era5_path)

#     #Navigation data
#     nav_folder = os.path.join(day_folder, 'Nav_files')

#     for x in os.listdir(nav_folder):
#         if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
#             os.remove(os.path.join(nav_folder, x))

#     nav_ict_path = os.path.join(nav_folder, os.listdir(nav_folder)[0])  #only one nav file per flight

#     if campaign != 'CPEX':  #campaign either CPEXAW or CPEXCV

#         nav_ict = icartt.Dataset(nav_ict_path)    #open the ict file with icartt
#         flight_lat = nav_ict.data["Latitude"]     #nav latitude, just a normal 1-D array
#         flight_lon = nav_ict.data["Longitude"]    #nav longitude, just a normal 1-D array

#     else:  #for CPEX navigation files, open the CSV file with pandas
#            #(Navigation files for CPEX (2017) are originally .kmz not .ict,
#            #so I converted them to CSV for free using https://www.gpsvisualizer.com/convert_input

#         nav_ict = pd.read_csv(nav_ict_path)       #open the ict file with pandas instead
#         flight_lat = nav_ict["latitude"].values   #nav latitude, just a normal 1-D array
#         flight_lon = nav_ict["longitude"].values  #nav longitude, just a normal 1-D array    

#     #flight track lat/lon extent [West,East,South,North] for plotting, giving an XX degree buffer around the flight track   
#     campaign_extent = [flight_lon.min() - 2.5, flight_lon.max() + 2.5, flight_lat.min() - 2.5, flight_lat.max() + 2.5]


#     ###calculate each near-storm dropsonde's mean lat/lon and add the sonde's time and mean lat/lon to a list to be plotted

#     drop_coords_and_time = []   #format: longitude, latitude, time (HHSS)

#     df_drop = pd.read_csv(drop_metric_filepath)
#     df_drop_use = df_drop[df_drop['Date'] == int(file_date)].copy()

#     for x in range(len(df_drop_use)):
#         date = str(df_drop_use['Date'].iloc[x])
#         time = str(df_drop_use['Time'].iloc[x]).zfill(6)
#         sonde_datetime = date[:4] + '-' + date[4:6] + '-' + date[6:] + ' ' + time[:2] + ':' + time[2:4] + ':' + time[4:]

#         drop_csv_use = drop_csv[drop_csv['Time [UTC]'] == sonde_datetime].copy()
#         drop_mean_lon = drop_csv_use['Longitude [deg]'].mean()
#         drop_mean_lat = drop_csv_use['Latitude [deg]'].mean()

#         sonde_info = [drop_mean_lon, drop_mean_lat, time[:4]]
#         drop_coords_and_time.append(sonde_info)

        
#     ###create an XX-panel plot of streamlines at XX-different pressure levels 
#         ###for each desired hour of the given day

#     half_pres_levs = math.ceil(len(pressures_to_plot_stream) / 2)   #to determine size of figure and number of subplots to generate (rounded up to the nearest multiple of 2)

#     for hr in utc_hours_to_plot:
        
#         hr2 = str(hr).zfill(2)
#         hour_prior = str(hr - 1).zfill(2)

#         #MIMIC TPW data
#            ##https://bin.ssec.wisc.edu/pub/mtpw2/data/
#         tpw_folder = os.path.join(day_folder, 'MIMIC_TPW_files')
#         tpw_path = os.path.join(tpw_folder, 'comp' + file_date + '.' + hr2 + '0000.nc')
#         ds_tpw = xr.open_dataset(tpw_path)

#         #GPM IMERG data (see IMERG.ipynb for more how to more generally download and plot IMERG data)
#            ##https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Read%20IMERG%20Data%20Using%20Python
#            ##https://disc.gsfc.nasa.gov/datasets?keywords=imerg&page=1
#            #0.1 x 0.1 gridded data, half-hourly means, using the half hour BEFORE the desired hour
#         imerg_folder = os.path.join(day_folder, 'IMERG_files')
        
#         for x in os.listdir(imerg_folder):
#             if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
#                 os.remove(os.path.join(imerg_folder, x))
                
#             #minutes and seconds automatically revert to zero (hour = 0, seconds = 0) for '%Y%m%d%H'
#             elif (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(seconds = 1), '%H%M%S') in x) and (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(minutes = 30), '%H%M%S') in x):
#                 imerg_file = x
#                 break
#             else:
#                 imerg_file = 'Could not find the desired IMERG file'
        
#         #confirm that the IMERG file is from the correct day (if hr2 == '00', then this will be the previous day)
#         assert (file_date in imerg_file) or (hr2 == '00'), 'IMERG file not from the correct day'
        
#         imerg_path = os.path.join(imerg_folder, imerg_file)
#         ds_imerg = h5py.File(imerg_path, 'r')
        
#         imerg_lons = ds_imerg['Grid/lon'][:]   #Longitude Shape: (3600,)
#         imerg_lats = ds_imerg['Grid/lat'][:]   #Latitude Shape: (1800,)
#         imerg_lons, imerg_lats = np.meshgrid(imerg_lons, imerg_lats)  #Long and lat grid shape: (1800, 3600) 
        
#         imerg_precip = ds_imerg['Grid/precipitation'][0][:][:]  #Original Precip Shape: (1, 3600, 1800) = (time, lon, lat)
#         imerg_precip = np.transpose(imerg_precip)               #New Precip Shape after transpose: (1800, 3600)
        
#         #mask blank data
#         imerg_precip_masked = np.ma.masked_where(imerg_precip < 0, imerg_precip)  #masks blank and bad data first (if blank data is -999 instead of NaN)
#         imerg_precip_masked = np.ma.masked_where(np.isnan(imerg_precip_masked), imerg_precip_masked)  #masks NaN values (not masked in previous line)        
        
#         data_proj = ccrs.PlateCarree()

#         group_fig = plt.figure(figsize = (12 * half_pres_levs, 5 * half_pres_levs))   #initialize the streamline figure for the given hour

#         for ii, pres_lev in enumerate(pressures_to_plot_stream):
#             ax = group_fig.add_subplot(2, half_pres_levs, ii+1, projection = data_proj)        

#             uwnd = ds.u.sel(time = file_date).sel(level = pres_lev)  #zonal winds (m/s)
#             uwnd = uwnd.sel(time = uwnd.time.dt.hour.isin(hr))       #zonal winds (m/s)

#             vwnd = ds.v.sel(time = file_date).sel(level = pres_lev)  #meridional winds (m/s)
#             vwnd = vwnd.sel(time = vwnd.time.dt.hour.isin(hr))       #meridional winds (m/s)
            
#             rh = ds.r.sel(time = file_date).sel(level = pres_lev)    #relative humidity (%)
#             rh = rh.sel(time = rh.time.dt.hour.isin(hr))             #relative humidity (%)

#             ##Smoothing (source: Hannah Zanowski) --> not recommended, see top of document
#                 ##Metpy smooth_n_point (data to be smoothed, number of points to use in smoothing (5 to 9 are valid), and number of times the smoother is applied)
#                     ##see https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.smooth_n_point.html for more info
#             #uwnd_smoothed = mpcalc.smooth_n_point(uwnd,9,10)
#             #vwnd_smoothed = mpcalc.smooth_n_point(vwnd,9,10)

#             ax.set_title('MIMIC TPW, GPM IMERG, and \n%i hPa Streamlines (%s, %s UTC)' % (pres_lev, file_date, hr2))
#             #ax.set_title('ERA5 %i hPa RH, GPM IMERG, and \n%i hPa Streamlines (%s, %s UTC)' % (pres_lev, pres_lev, file_date, hr2))
#             ax.set_extent(campaign_extent, ccrs.PlateCarree()) #lat/lon bounds are [West,East,South,North]

#             # Add land, coastlines, and borders
#             #ax.add_feature(cfeature.LAND, facecolor='0.8')
#             ax.coastlines(ls = '-', linewidth = 0.5, color = 'gray')
            
#             #plot MIMIC TPW
#             tpw_levels = np.arange(0, 70.5, 2)
#             pm0 = ax.contourf(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, levels = tpw_levels,
#                               extend = 'max', cmap = cm.jet, transform = data_proj)
# #             pm0 = ax.pcolormesh(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, vmin = 0, vmax = 70,
# #                                 cmap = cm.jet, transform = data_proj, zorder = 0)

#         #     #plot ERA5 RH
#         #     tpw_levels = np.arange(0, 100.1, 2)  #actually RH levels, but keeping the tpw_levels name because we use it elsewhere
#         #     pm0 = ax.contourf(ds.longitude, ds.latitude, rh[0].values, levels = tpw_levels,
#         #                       extend = 'max', cmap = cm.jet, transform = data_proj, alpha = 0.6)
#         # #             pm0 = ax.pcolormesh(ds.longitude, ds.latitude, rh[0].values, vmin = 0, vmax = 70,
#         # #                                 cmap = cm.jet, transform = data_proj, zorder = 0)
    
#             #plot IMERG Rain Rate
#             pm1 = ax.contourf(imerg_lons, imerg_lats, imerg_precip_masked, 
#                               levels = np.logspace(np.log10(0.1), np.log10(40), num = len(tpw_levels)), 
#                               norm = 'log', extend = 'max', 
#                               cmap = cm.jet, transform = data_proj, zorder = 1)
# #             pm1 = ax.pcolormesh(imerg_lons, imerg_lats, imerg_precip_masked, 
# #                                 norm = mplc.LogNorm(vmin = 0.1, vmax = 40), 
# #                                 cmap = cm.jet, transform = data_proj, zorder = 1) 

#             #Gridlines
#             gl = ax.gridlines(crs = ccrs.PlateCarree(), draw_labels = True, 
#                               linewidth = 0.5, color = 'gray', alpha = 0.5, linestyle = '--', zorder = 2)
#             gl.top_labels = False
#             gl.right_labels = False

#             #plot ERA5 streamlines
#             ax.streamplot(ds.longitude, ds.latitude, uwnd[0].values, vwnd[0].values,
#                           color = 'k', linewidth = 0.6, density = 1.0, transform = data_proj, zorder = 3)

#             #plot flight track
#             ax.plot(flight_lon, flight_lat, color = 'white', linewidth = 1.5, zorder = 4)            
            
#             #plot near-storm dropsonde locations for the given flight 
#                 #if the dropsonde was deployed within 30 minutes (1-hr total range) of the given hour
#                     #NOTE: Dropsondes with no wind data don't have GPS data either (5 of them total)
#             for sonde in drop_coords_and_time:
#                 if (sonde[2][:2] == hour_prior and int(sonde[2][2:4]) >= 30) or (sonde[2][:2] == hr2 and int(sonde[2][2:4]) < 30):
#                     #ax.scatter(sonde[0], sonde[1], marker = f'${sonde[2]}$', color = 'b', s = 300)
#                     ax.scatter(sonde[0], sonde[1], marker = '*', color = 'k', zorder = 5, s = 300)

#             #same as above, but labeling the dropsondes by the order that they appear in the
#                 #Dropsonde_Metric_Calculations.csv, NOT IN CHRONOLOGICAL ORDER!!!
#             #for z, sonde in enumerate(drop_coords_and_time):
#                 #ax.scatter(sonde[0], sonde[1], marker = f'${z + 1}$', color = 'b', s = 120, zorder = 5) 
                
#             #plotting the colorbars
#             #cbar0 = group_fig.colorbar(pm0, ax = ax, orientation = 'vertical', shrink = 0.75, pad = 0.25)
#             #cbar0.set_label('TPW [mm]')
#             #cbar0.ax.yaxis.set_ticks_position('left')
#             #cbar0.ax.yaxis.set_label_position('left')
            
#             #this works with GeoAxes (i.e., Cartopy's map projections)
#             if ii == len(pressures_to_plot_stream) - 1:
                
#                 #MIMIC TPW colorbar
#                 ticks_tpw = np.arange(0, 70.5, 10, dtype = int)
#                 #cax = group_fig.add_axes([ax.get_position().x1+0.05, ax.get_position().y0, 0.02, ax.get_position().height])
#                 cax0 = group_fig.add_axes([ax.get_position().x1 + 0.05, ax.get_position().y0, 0.02, 0.75])
#                 cbar0 = group_fig.colorbar(pm0, cax = cax0, ticks = ticks_tpw)
#                 cbar0.set_label('TPW [mm]')
#                 cbar0.ax.set_yticklabels(list(map(str, list(ticks_tpw))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
#                 cbar0.ax.yaxis.set_ticks_position('left')
#                 cbar0.ax.yaxis.set_label_position('left')

#                 # #ERA5 RH colorbar
#                 # ticks_rh = np.arange(0, 100.5, 10, dtype = int)
#                 # #cax = group_fig.add_axes([ax.get_position().x1+0.05, ax.get_position().y0, 0.02, ax.get_position().height])
#                 # cax0 = group_fig.add_axes([ax.get_position().x1 + 0.05, ax.get_position().y0, 0.02, 0.75])
#                 # cbar0 = group_fig.colorbar(pm0, cax = cax0, ticks = ticks_rh)
#                 # cbar0.set_label('Relative Humidity [%]')
#                 # cbar0.ax.set_yticklabels(list(map(str, list(ticks_rh))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
#                 # cbar0.ax.yaxis.set_ticks_position('left')
#                 # cbar0.ax.yaxis.set_label_position('left')
                
#                 #IMERG colorbar
#                 ticks_imerg = np.array([0.1, 1, 5, 10, 20, 40], dtype = float)
#                 cax1 = group_fig.add_axes([ax.get_position().x1 + 0.05, ax.get_position().y0, 0.02, 0.75])
#                 cbar1 = group_fig.colorbar(pm1, cax = cax1, ticks = ticks_imerg)
#                 cbar1.set_label('IMERG [mm hr$\\bf{^{-1}}$]')
#                 cbar1.ax.set_yticklabels(list(map(str, list(ticks_imerg))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
#                 cbar1.ax.yaxis.set_ticks_position('right')
#                 cbar1.ax.yaxis.set_label_position('right')
                
#         ds_tpw.close()
#         ds_imerg.close()                
                
#         #plt.tight_layout()
#         #plt.subplots_adjust(wspace = 0.1)

#         #save the figure
#         #plot_save_name = f'ERA5_streamlines_RH_{hr2}UTC.png'
#         #plot_save_name = f'ERA5_streamlines_TPW_{hr2}UTC.png'
#         plot_save_name = f'ERA5_precip_TPW_{hr2}UTC.png'
#         plt.savefig(os.path.join(day_folder, plot_save_name), bbox_inches = 'tight')
#         #plt.show()  #plt.show() must come after plt.savefig() in order for the image to save properly
#         #plt.clf()   #supposedly speeds things up? According to: https://www.youtube.com/watch?v=jGVIZbi9uMY
#         plt.close()
#         plt.clf()    #if placing this after plt.close(), may release memory related to the figure (https://stackoverflow.com/questions/741877/how-do-i-tell-matplotlib-that-i-am-done-with-a-plot)

#         ##decrease file size of the image by 66% without noticeable image effects (if using Matplotlib)
#         ##(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
#         im = Image.open(os.path.join(day_folder, plot_save_name))

#         try:
#             im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
#         except:
#             #use this for older version of PIL/Pillow if the above line doesn't work, 
#             #though this line will have isolated, extremely minor image effects due to 
#             #only using 256 colors instead of the 3-element RGB scale
#             im2 = im.convert('P')

#         im2.save(os.path.join(day_folder, plot_save_name))
#         im.close()
#         im2.close()

#     ds.close()
    
#     print (file_date + ' streamline plots complete!\n')


#### Convergence at Various Pressure Levels with flight track and dropsondes overlaid for each desired flight and time ranges in case_dict_conv

In [ ]:
# #set some baseline plot displays

# #matplotlib.rcParams['axes.facecolor'] = [0.9,0.9,0.9]
# matplotlib.rcParams['axes.labelsize'] = 12
# matplotlib.rcParams['axes.titlesize'] = 12
# matplotlib.rcParams['axes.labelweight'] = 'bold'
# matplotlib.rcParams['axes.titleweight'] = 'bold'
# matplotlib.rcParams['xtick.labelsize'] = 12
# matplotlib.rcParams['ytick.labelsize'] = 12
# matplotlib.rcParams['legend.fontsize'] = 16
# #matplotlib.rcParams['legend.facecolor'] = 'w'
# #matplotlib.rcParams['axes.facecolor'] = 'w'
# matplotlib.rcParams['font.family'] = 'arial'
# matplotlib.rcParams['hatch.linewidth'] = 0.3

# #file_date is the date on which the desired flight took place
# #utc_hours_to_plot is the desired UTC hours to be plotted for the given flight (e.g., convective case hours/time range)
# for file_date, utc_hours_to_plot in case_dict_conv.items():

#     print (file_date + ' convergence plots in progress...')
    
#     ###get locations of the dropsonde/Navigation/ERA5 folder and read the appropriate files in
#     day_folder = os.path.join(os.getcwd(), file_date)

#     #dropsonde data
#     drop_csv_path = os.path.join(day_folder, 'final_dropsonde_' + file_date + '.csv')
#     drop_csv = pd.read_csv(drop_csv_path)

#     if file_date[:4] == '2017':
#         campaign = 'CPEX'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
#     elif file_date[:4] == '2021':
#         campaign = 'CPEXAW'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
#     elif file_date[:4] == '2022':
#         campaign = 'CPEXCV'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')
#     else:
#         pass

#     #ERA5 data
#     era5_folder = os.path.join(os.getcwd(), 'ERA5_Reanalysis_Data')
#     era5_path = os.path.join(era5_folder, campaign + '_ERA5_Reanalysis_Hourly_Pressure.nc')
#     ds = xr.open_dataset(era5_path)

#     #Navigation data
#     nav_folder = os.path.join(day_folder, 'Nav_files')

#     for x in os.listdir(nav_folder):
#         if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
#             os.remove(os.path.join(nav_folder, x))

#     nav_ict_path = os.path.join(nav_folder, os.listdir(nav_folder)[0])  #only one nav file per flight

#     if campaign != 'CPEX':  #campaign either CPEXAW or CPEXCV

#         nav_ict = icartt.Dataset(nav_ict_path)    #open the ict file with icartt
#         flight_lat = nav_ict.data["Latitude"]     #nav latitude, just a normal 1-D array
#         flight_lon = nav_ict.data["Longitude"]    #nav longitude, just a normal 1-D array

#     else:  #for CPEX navigation files, open the CSV file with pandas
#            #(Navigation files for CPEX (2017) are originally .kmz not .ict,
#            #so I converted them to CSV for free using https://www.gpsvisualizer.com/convert_input

#         nav_ict = pd.read_csv(nav_ict_path)       #open the ict file with pandas instead
#         flight_lat = nav_ict["latitude"].values   #nav latitude, just a normal 1-D array
#         flight_lon = nav_ict["longitude"].values  #nav longitude, just a normal 1-D array    

#     #flight track lat/lon extent [West,East,South,North] for plotting, giving an XX degree buffer around the flight track   
#     campaign_extent = [flight_lon.min() - 2.5, flight_lon.max() + 2.5, flight_lat.min() - 2.5, flight_lat.max() + 2.5]


#     ###calculate each near-storm dropsonde's mean lat/lon and add the sonde's time and mean lat/lon to a list to be plotted

#     drop_coords_and_time = []   #format: longitude, latitude, time (HHSS)

#     df_drop = pd.read_csv(drop_metric_filepath)
#     df_drop_use = df_drop[df_drop['Date'] == int(file_date)].copy()

#     for x in range(len(df_drop_use)):
#         date = str(df_drop_use['Date'].iloc[x])
#         time = str(df_drop_use['Time'].iloc[x]).zfill(6)
#         sonde_datetime = date[:4] + '-' + date[4:6] + '-' + date[6:] + ' ' + time[:2] + ':' + time[2:4] + ':' + time[4:]

#         drop_csv_use = drop_csv[drop_csv['Time [UTC]'] == sonde_datetime].copy()
#         drop_mean_lon = drop_csv_use['Longitude [deg]'].mean()
#         drop_mean_lat = drop_csv_use['Latitude [deg]'].mean()

#         sonde_info = [drop_mean_lon, drop_mean_lat, time[:4]]
#         drop_coords_and_time.append(sonde_info)

        
#     ###create an XX-panel plot of convergence at XX-different pressure levels 
#         ###for each desired hour of the given day

#     half_pres_levs = math.ceil(len(pressures_to_plot_conv) / 2)   #to determine size of figure and number of subplots to generate (rounded up to the nearest multiple of 2)

#     for hr in utc_hours_to_plot:
        
#         hr2 = str(hr).zfill(2)
#         hour_prior = str(hr - 1).zfill(2)

#         #MIMIC TPW data
#             ##https://bin.ssec.wisc.edu/pub/mtpw2/data/
#         tpw_folder = os.path.join(day_folder, 'MIMIC_TPW_files')
#         tpw_path = os.path.join(tpw_folder, 'comp' + file_date + '.' + hr2 + '0000.nc')
#         ds_tpw = xr.open_dataset(tpw_path)

#         #GPM IMERG data (see IMERG.ipynb for more how to more generally download and plot IMERG data)
#            ##https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Read%20IMERG%20Data%20Using%20Python
#            ##https://disc.gsfc.nasa.gov/datasets?keywords=imerg&page=1
#            #0.1 x 0.1 gridded data, half-hourly means, using the half hour BEFORE the desired hour
#         imerg_folder = os.path.join(day_folder, 'IMERG_files')
        
#         for x in os.listdir(imerg_folder):
#             if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
#                 os.remove(os.path.join(imerg_folder, x))
                
#             #minutes and seconds automatically revert to zero (hour = 0, seconds = 0) for '%Y%m%d%H'
#             elif (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(seconds = 1), '%H%M%S') in x) and (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(minutes = 30), '%H%M%S') in x):
#                 imerg_file = x
#                 break
#             else:
#                 imerg_file = 'Could not find the desired IMERG file'
        
#         #confirm that the IMERG file is from the correct day (if hr2 == '00', then this will be the previous day)
#         assert (file_date in imerg_file) or (hr2 == '00'), 'IMERG file not from the correct day'
        
#         imerg_path = os.path.join(imerg_folder, imerg_file)
#         ds_imerg = h5py.File(imerg_path, 'r')
        
#         imerg_lons = ds_imerg['Grid/lon'][:]   #Longitude Shape: (3600,)
#         imerg_lats = ds_imerg['Grid/lat'][:]   #Latitude Shape: (1800,)
#         imerg_lons, imerg_lats = np.meshgrid(imerg_lons, imerg_lats)  #Long and lat grid shape: (1800, 3600) 
        
#         imerg_precip = ds_imerg['Grid/precipitation'][0][:][:]  #Original Precip Shape: (1, 3600, 1800) = (time, lon, lat)
#         imerg_precip = np.transpose(imerg_precip)               #New Precip Shape after transpose: (1800, 3600)
        
#         #mask blank data
#         imerg_precip_masked = np.ma.masked_where(imerg_precip < 0, imerg_precip)  #masks blank and bad data first (if blank data is -999 instead of NaN)
#         imerg_precip_masked = np.ma.masked_where(np.isnan(imerg_precip_masked), imerg_precip_masked)  #masks NaN values (not masked in previous line)
        
#         data_proj = ccrs.PlateCarree()

#         group_fig = plt.figure(figsize = (6 * half_pres_levs, 6 * half_pres_levs))
        
#         for ii, pres_lev in enumerate(pressures_to_plot_conv):
#             ax = group_fig.add_subplot(2, half_pres_levs, ii+1, projection = data_proj)

#             conv = ds.d.sel(time = file_date).sel(level = pres_lev) * -1   #convergence of the wind (1/s)
#             conv = conv.sel(time = conv.time.dt.hour.isin(hr)) * 10**5     #convergence of the wind (times 10**5 1/s)
            
#             rh = ds.r.sel(time = file_date).sel(level = pressures_to_plot_RH[ii])    #relative humidity (%)
#             rh = rh.sel(time = rh.time.dt.hour.isin(hr))                             #relative humidity (%)

#             ##Smoothing (source: Hannah Zanowski) --> not recommended, see top of document
#                 ##Metpy smooth_n_point (data to be smoothed, number of points to use in smoothing (5 to 9 are valid), and number of times the smoother is applied)
#                     ##see https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.smooth_n_point.html for more info
#             #conv_smoothed = mpcalc.smooth_n_point(conv,9,1)
    
#             #ax.set_title('MIMIC TPW, GPM IMERG, and \n%i hPa ERA5 Convergence ($\\bf{10^{-5} s^{-1}}$) (%s, %s UTC)' % (pres_lev, file_date, hr2))
#             ax.set_title('ERA5 %i hPa RH, GPM IMERG, and \nERA5 %i hPa Convergence ($\\bf{10^{-5} s^{-1}}$) (%s, %s UTC)' % (pressures_to_plot_RH[ii], pres_lev, file_date, hr2))
#             ax.set_extent(campaign_extent, ccrs.PlateCarree()) #lat/lon bounds are [West,East,South,North]

#             # Add land, coastlines, and borders
#             #ax.add_feature(cfeature.LAND, facecolor='0.8')
#             ax.coastlines(ls = '-', linewidth = 0.5, color = 'gray')

# #             #plot MIMIC TPW
# #             tpw_levels = np.arange(0, 70.5, 2)
# #             pm0 = ax.contourf(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, levels = tpw_levels,
# #                               extend = 'max', cmap = cm.jet, transform = data_proj)
# # #             pm0 = ax.pcolormesh(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, vmin = 0, vmax = 70,
# # #                                 cmap = cm.jet, transform = data_proj, zorder = 0)

#             #plot ERA5 RH
#             tpw_levels = np.arange(0, 100.1, 2)  #actually RH levels, but keeping the tpw_levels name because we use it elsewhere
#             pm0 = ax.contourf(ds.longitude, ds.latitude, rh[0].values, levels = tpw_levels,
#                               extend = 'max', cmap = cm.jet, transform = data_proj, alpha = 0.6)
#         #             pm0 = ax.pcolormesh(ds.longitude, ds.latitude, rh[0].values, vmin = 0, vmax = 70,
#         #                                 cmap = cm.jet, transform = data_proj, zorder = 0)
    
#             #plot IMERG Rain Rate
#             pm1 = ax.contourf(imerg_lons, imerg_lats, imerg_precip_masked, 
#                               levels = np.logspace(np.log10(0.1), np.log10(40), num = len(tpw_levels)), 
#                               norm = 'log', extend = 'max', 
#                               cmap = cm.jet, transform = data_proj, zorder = 1)
# #             pm1 = ax.pcolormesh(imerg_lons, imerg_lats, imerg_precip_masked, 
# #                                 norm = mplc.LogNorm(vmin = 0.1, vmax = 40), 
# #                                 cmap = cm.jet, transform = data_proj, zorder = 1)
            
#             #gridlines
#             gl = ax.gridlines(crs = ccrs.PlateCarree(), draw_labels = True, 
#                               linewidth = 0.5, color = 'gray', alpha = 0.5, linestyle = '--', zorder = 2)
#             gl.top_labels = False
#             gl.right_labels = False

#             #plot ERA5 convergence
#             ax.contour(ds.longitude, ds.latitude, conv[0].values, levels = np.arange(1, 201.1, 2),
#                        colors = 'k', linewidths = 0.6, transform = data_proj, zorder = 3)            
            
#             #plot flight track
#             ax.plot(flight_lon, flight_lat, color = 'white', linewidth = 1.5, zorder = 4)
                
#             #plot near-storm dropsonde locations for the given flight 
#                 #if the dropsonde was deployed within 30 minutes (1-hr total range) of the given hour
#                     #NOTE: Dropsondes with no wind data don't have GPS data either (5 of them total)
#             for sonde in drop_coords_and_time:
#                 if (sonde[2][:2] == hour_prior and int(sonde[2][2:4]) >= 30) or (sonde[2][:2] == hr2 and int(sonde[2][2:4]) < 30):
#                     #ax.scatter(sonde[0], sonde[1], marker = f'${sonde[2]}$', color = 'b', s = 300)
#                     ax.scatter(sonde[0], sonde[1], marker = '*', color = 'k', s = 80, zorder = 5)
                
#             #same as above, but labeling the dropsondes by the order that they appear in the
#                 #Dropsonde_Metric_Calculations.csv, NOT IN CHRONOLOGICAL ORDER!!!
#             #for z, sonde in enumerate(drop_coords_and_time):
#                 #ax.scatter(sonde[0], sonde[1], marker = f'${z + 1}$', color = 'b', s = 120, zorder = 5)

#             #plotting the colorbars
#             #cbar0 = group_fig.colorbar(pm0, ax = ax, orientation = 'vertical', shrink = 0.75, pad = 0.25)
#             #cbar0.set_label('TPW [mm]')
#             #cbar0.ax.yaxis.set_ticks_position('left')
#             #cbar0.ax.yaxis.set_label_position('left')
            
#             #this works with GeoAxes (i.e., Cartopy's map projections)
#             if ii == len(pressures_to_plot_conv) - 1:
                
# #                 #MIMIC TPW colorbar
# #                 ticks_tpw = np.arange(0, 70.5, 10, dtype = int)
# #                 #cax = group_fig.add_axes([ax.get_position().x1+0.05, ax.get_position().y0, 0.02, ax.get_position().height])
# #                 cax0 = group_fig.add_axes([ax.get_position().x1 + 0.1, ax.get_position().y0, 0.02, 0.75])
# #                 cbar0 = group_fig.colorbar(pm0, cax = cax0, ticks = ticks_tpw)
# #                 cbar0.set_label('TPW [mm]')
# #                 cbar0.ax.set_yticklabels(list(map(str, list(ticks_tpw))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
# #                 cbar0.ax.yaxis.set_ticks_position('left')
# #                 cbar0.ax.yaxis.set_label_position('left')

#                 #ERA5 RH colorbar
#                 ticks_rh = np.arange(0, 100.5, 10, dtype = int)
#                 #cax = group_fig.add_axes([ax.get_position().x1+0.05, ax.get_position().y0, 0.02, ax.get_position().height])
#                 cax0 = group_fig.add_axes([ax.get_position().x1 + 0.1, ax.get_position().y0, 0.02, 0.75])
#                 cbar0 = group_fig.colorbar(pm0, cax = cax0, ticks = ticks_rh)
#                 cbar0.set_label('Relative Humidity [%]')
#                 cbar0.ax.set_yticklabels(list(map(str, list(ticks_rh))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
#                 cbar0.ax.yaxis.set_ticks_position('left')
#                 cbar0.ax.yaxis.set_label_position('left')
                
#                 #IMERG colorbar
#                 ticks_imerg = np.array([0.1, 1, 5, 10, 20, 40], dtype = float)
#                 cax1 = group_fig.add_axes([ax.get_position().x1 + 0.1, ax.get_position().y0, 0.02, 0.75])
#                 cbar1 = group_fig.colorbar(pm1, cax = cax1, ticks = ticks_imerg)
#                 cbar1.set_label('IMERG [mm hr$\\bf{^{-1}}$]')
#                 cbar1.ax.set_yticklabels(list(map(str, list(ticks_imerg))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
#                 cbar1.ax.yaxis.set_ticks_position('right')
#                 cbar1.ax.yaxis.set_label_position('right')
                
#         ds_tpw.close()
#         ds_imerg.close()
            
#         #plt.tight_layout()
#         plt.subplots_adjust(wspace = 0.3)

#         #save the figure
#         plot_save_name = f'ERA5_convergence_midRH_{hr2}UTC.png'
#         plt.savefig(os.path.join(day_folder, plot_save_name), bbox_inches = 'tight')
#         #plt.show()  #plt.show() must come after plt.savefig() in order for the image to save properly
#         #plt.clf()   #supposedly speeds things up? According to: https://www.youtube.com/watch?v=jGVIZbi9uMY
#         plt.close()
#         plt.clf()    #if placing this after plt.close(), may release memory related to the figure (https://stackoverflow.com/questions/741877/how-do-i-tell-matplotlib-that-i-am-done-with-a-plot)

#         ##decrease file size of the image by 66% without noticeable image effects (if using Matplotlib)
#         ##(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
#         im = Image.open(os.path.join(day_folder, plot_save_name))

#         try:
#             im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
#         except:
#             #use this for older version of PIL/Pillow if the above line doesn't work, 
#             #though this line will have isolated, extremely minor image effects due to 
#             #only using 256 colors instead of the 3-element RGB scale
#             im2 = im.convert('P')

#         im2.save(os.path.join(day_folder, plot_save_name))
#         im.close()
#         im2.close()

#     ds.close()
    
#     print (file_date + ' convergence plots complete!\n')


#### Figure 11 for AGU Paper (2023)

In [ ]:
# #FIGURE 11 FOR AGU PAPER (2023)

# case_dict_agu = {0: ['20170611', 'Case 13', 19, [-95, -80, 20, 28]], 
#                  1: ['20210824', 'Case 16', 19, [-75, -60, 10, 18]]}  #flight track lat/lon extent [West,East,South,North]
# conv_pres = 975  #hPa; pressure level to plot ERA5 convergence for

# drop_metric_cases1316_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_OnlyCases13and16.csv')
# df_drop_cases1316 = pd.read_csv(drop_metric_cases1316_filepath)

# #set some baseline plot displays

# #matplotlib.rcParams['axes.facecolor'] = [0.9,0.9,0.9]
# matplotlib.rcParams['axes.labelsize'] = 16
# matplotlib.rcParams['axes.titlesize'] = 19
# matplotlib.rcParams['axes.labelweight'] = 'bold'
# matplotlib.rcParams['axes.titleweight'] = 'bold'
# matplotlib.rcParams['xtick.labelsize'] = 16
# matplotlib.rcParams['ytick.labelsize'] = 16
# matplotlib.rcParams['legend.fontsize'] = 16
# #matplotlib.rcParams['legend.facecolor'] = 'w'
# #matplotlib.rcParams['axes.facecolor'] = 'w'
# matplotlib.rcParams['font.family'] = 'arial'
# matplotlib.rcParams['hatch.linewidth'] = 0.3

# data_proj = ccrs.PlateCarree()
# group_fig = plt.figure(figsize = (30,15))

# for key, case_info in case_dict_agu.items():
    
#     file_date = case_info[0]
#     case_num = case_info[1]   
#     hr = case_info[2]
#     campaign_extent = case_info[3]

#     print (file_date + ' convergence plots in progress...')
    
#     ###get locations of the dropsonde/Navigation/ERA5 folder and read the appropriate files in
#     day_folder = os.path.join(os.getcwd(), file_date)

#     #dropsonde data
#     drop_csv_path = os.path.join(day_folder, 'final_dropsonde_' + file_date + '.csv')
#     drop_csv = pd.read_csv(drop_csv_path)

#     if file_date[:4] == '2017':
#         campaign = 'CPEX'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
#     elif file_date[:4] == '2021':
#         campaign = 'CPEXAW'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
#     elif file_date[:4] == '2022':
#         campaign = 'CPEXCV'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')
#     else:
#         pass


#     #ERA5 data
#     era5_folder = os.path.join(os.getcwd(), 'ERA5_Reanalysis_Data')
#     era5_path = os.path.join(era5_folder, campaign + '_ERA5_Reanalysis_Hourly_Pressure.nc')
#     ds = xr.open_dataset(era5_path)

#     #Navigation data
#     nav_folder = os.path.join(day_folder, 'Nav_files')

#     for x in os.listdir(nav_folder):
#         if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
#             os.remove(os.path.join(nav_folder, x))

#     nav_ict_path = os.path.join(nav_folder, os.listdir(nav_folder)[0])  #only one nav file per flight

#     if campaign != 'CPEX':  #campaign either CPEXAW or CPEXCV

#         nav_ict = icartt.Dataset(nav_ict_path)    #open the ict file with icartt
#         flight_lat = nav_ict.data["Latitude"]     #nav latitude, just a normal 1-D array
#         flight_lon = nav_ict.data["Longitude"]    #nav longitude, just a normal 1-D array

#     else:  #for CPEX navigation files, open the CSV file with pandas
#            #(Navigation files for CPEX (2017) are originally .kmz not .ict,
#            #so I converted them to CSV for free using https://www.gpsvisualizer.com/convert_input

#         nav_ict = pd.read_csv(nav_ict_path)       #open the ict file with pandas instead
#         flight_lat = nav_ict["latitude"].values   #nav latitude, just a normal 1-D array
#         flight_lon = nav_ict["longitude"].values  #nav longitude, just a normal 1-D array    


#     ###calculate each near-storm dropsonde's mean lat/lon and add the sonde's time and mean lat/lon to a list to be plotted

#     drop_coords_and_time = []   #format: longitude, latitude, time (HHSS)

#     df_drop = pd.read_csv(drop_metric_filepath)
#     df_drop_use = df_drop[df_drop['Date'] == int(file_date)].copy()

#     for x in range(len(df_drop_use)):
#         date = str(df_drop_use['Date'].iloc[x])
#         time = str(df_drop_use['Time'].iloc[x]).zfill(6)
#         sonde_datetime = date[:4] + '-' + date[4:6] + '-' + date[6:] + ' ' + time[:2] + ':' + time[2:4] + ':' + time[4:]

#         drop_csv_use = drop_csv[drop_csv['Time [UTC]'] == sonde_datetime].copy()
#         drop_mean_lon = drop_csv_use['Longitude [deg]'].mean()
#         drop_mean_lat = drop_csv_use['Latitude [deg]'].mean()

#         sonde_info = [drop_mean_lon, drop_mean_lat, time[:4], date, time]
#         drop_coords_and_time.append(sonde_info)
        
#     hr2 = str(hr).zfill(2)

#     #MIMIC TPW data
#         ##https://bin.ssec.wisc.edu/pub/mtpw2/data/
#     tpw_folder = os.path.join(day_folder, 'MIMIC_TPW_files')
#     tpw_path = os.path.join(tpw_folder, 'comp' + file_date + '.' + hr2 + '0000.nc')
#     ds_tpw = xr.open_dataset(tpw_path)

#     #GPM IMERG data (see IMERG.ipynb for more how to more generally download and plot IMERG data)
#         ##https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Read%20IMERG%20Data%20Using%20Python
#         ##https://disc.gsfc.nasa.gov/datasets?keywords=imerg&page=1
#         #0.1 x 0.1 gridded data, half-hourly means, using the half hour BEFORE the desired hour
#     imerg_folder = os.path.join(day_folder, 'IMERG_files')

#     for x in os.listdir(imerg_folder):
#         if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
#             os.remove(os.path.join(imerg_folder, x))

#         #minutes and seconds automatically revert to zero (hour = 0, seconds = 0) for '%Y%m%d%H'
#         elif (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(seconds = 1), '%H%M%S') in x) and (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(minutes = 30), '%H%M%S') in x):
#             imerg_file = x
#             break
#         else:
#             imerg_file = 'Could not find the desired IMERG file'

#     #confirm that the IMERG file is from the correct day (if hr2 == '00', then this will be the previous day)
#     assert (file_date in imerg_file) or (hr2 == '00'), 'IMERG file not from the correct day'

#     imerg_path = os.path.join(imerg_folder, imerg_file)
#     ds_imerg = h5py.File(imerg_path, 'r')

#     imerg_lons = ds_imerg['Grid/lon'][:]   #Longitude Shape: (3600,)
#     imerg_lats = ds_imerg['Grid/lat'][:]   #Latitude Shape: (1800,)
#     imerg_lons, imerg_lats = np.meshgrid(imerg_lons, imerg_lats)  #Long and lat grid shape: (1800, 3600) 

#     imerg_precip = ds_imerg['Grid/precipitation'][0][:][:]  #Original Precip Shape: (1, 3600, 1800) = (time, lon, lat)
#     imerg_precip = np.transpose(imerg_precip)               #New Precip Shape after transpose: (1800, 3600)

#     #mask blank data
#     imerg_precip_masked = np.ma.masked_where(imerg_precip < 0, imerg_precip)  #masks blank and bad data first (if blank data is -999 instead of NaN)
#     imerg_precip_masked = np.ma.masked_where(np.isnan(imerg_precip_masked), imerg_precip_masked)  #masks NaN values (not masked in previous line)

#     #creat the plot
#     ax = group_fig.add_subplot(2, 1, key+1, projection = data_proj)

#     conv = ds.d.sel(time = file_date).sel(level = conv_pres) * -1   #convergence of the wind (1/s)
#     conv = conv.sel(time = conv.time.dt.hour.isin(hr)) * 10**5      #convergence of the wind (times 10**5 1/s)

#     ##Smoothing (source: Hannah Zanowski) --> not recommended, see top of document
#         ##Metpy smooth_n_point (data to be smoothed, number of points to use in smoothing (5 to 9 are valid), and number of times the smoother is applied)
#             ##see https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.smooth_n_point.html for more info
#     #conv_smoothed = mpcalc.smooth_n_point(conv,9,1)

#     ax.set_title('%s MIMIC TPW, GPM IMERG, and ERA5 %i hPa Convergence (%s UTC)' % (case_num, conv_pres, hr2))
#     ax.set_extent(campaign_extent, ccrs.PlateCarree()) #lat/lon bounds are [West,East,South,North]

#     # Add land, coastlines, and borders
#     #ax.add_feature(cfeature.LAND, facecolor='0.8')
#     ax.coastlines(ls = '-', linewidth = 0.5, color = 'gray')

#     #plot MIMIC TPW
#     tpw_levels = np.arange(0, 70.5, 2)
#     pm0 = ax.contourf(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, levels = tpw_levels,
#                       extend = 'max', cmap = cm.jet, transform = data_proj)
# #             pm0 = ax.pcolormesh(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, vmin = 0, vmax = 70,
# #                                 cmap = cm.jet, transform = data_proj, zorder = 0)

#     #plot IMERG Rain Rate
#     pm1 = ax.contourf(imerg_lons, imerg_lats, imerg_precip_masked, 
#                       levels = np.logspace(np.log10(0.1), np.log10(40), num = len(tpw_levels)), 
#                       norm = 'log', extend = 'max', 
#                       cmap = cm.jet, transform = data_proj, zorder = 1)
# #             pm1 = ax.pcolormesh(imerg_lons, imerg_lats, imerg_precip_masked, 
# #                                 norm = mplc.LogNorm(vmin = 0.1, vmax = 40), 
# #                                 cmap = cm.jet, transform = data_proj, zorder = 1)

#     #gridlines
#     gl = ax.gridlines(crs = ccrs.PlateCarree(), draw_labels = True, linewidth = 0.5, 
#                       color = 'gray', alpha = 0.5, linestyle = '--', zorder = 2)
#     gl.top_labels = False
#     gl.right_labels = False
#     gl.xlabel_style = {'size':16, 'color':'black'}
#     gl.ylabel_style = {'size':16, 'color':'black'}
    
#     level_array = np.arange(1, 201.1, 2)
#     linewidth_array = np.full_like(level_array, 0.6)
#     #linewidth_array[3:] = 2.5   #values 7 x 10^-5 s^-1 and greater will have thicker contours
#     linewidth_array[2:] = 2.5   #values 5 x 10^-5 s^-1 and greater will have thicker contours

#     #plot ERA5 convergence
#     ax.contour(ds.longitude, ds.latitude, conv[0].values, levels = level_array,
#                colors = 'k', linewidths = linewidth_array, transform = data_proj, zorder = 3)            

#     #plot flight track
#     ax.plot(flight_lon, flight_lat, color = 'darkmagenta', linewidth = 2.5, zorder = 4)

#     #plot near-storm dropsonde locations for the given flight
#         #NOTE: Dropsondes with no wind data don't have GPS data either (5 of them total)
#     for sonde in drop_coords_and_time:
#         #ax.scatter(sonde[0], sonde[1], marker = f'${sonde[2]}$', color = 'b', s = 300)
#         df_drop_cases1316_use = df_drop_cases1316[(df_drop_cases1316['Date'] == int(sonde[3])) & (df_drop_cases1316['Time'] == int(sonde[4]))].copy()
        
#         #skip plotting dropsondes from the given flight that aren't associated with Case 13 nor Case 16
#         if len(df_drop_cases1316_use) == 0:
#             continue

#         if df_drop_cases1316_use['Beyond TPW Gradient'].iloc[0] == 'Yes':
#             ax.scatter(sonde[0], sonde[1], marker = '^', color = 'k', zorder = 5, s = 250)
#         else:
#             ax.scatter(sonde[0], sonde[1], marker = '*', color = 'k', zorder = 5, s = 400)
        

#     #same as above, but labeling the dropsondes by the order that they appear in the
#         #Dropsonde_Metric_Calculations.csv, NOT IN CHRONOLOGICAL ORDER!!!
#     #for z, sonde in enumerate(drop_coords_and_time):
#         #ax.scatter(sonde[0], sonde[1], marker = f'${z + 1}$', color = 'b', s = 120, zorder = 5)

#     #plotting the colorbars
#     #cbar0 = group_fig.colorbar(pm0, ax = ax, orientation = 'vertical', shrink = 0.75, pad = 0.25)
#     #cbar0.set_label('TPW [mm]')
#     #cbar0.ax.yaxis.set_ticks_position('left')
#     #cbar0.ax.yaxis.set_label_position('left')

#     #this works with GeoAxes (i.e., Cartopy's map projections)
#     if key == len(case_dict_agu) - 1:

#         #MIMIC TPW colorbar
#         ticks_tpw = np.arange(0, 70.5, 10)
#         #cax = group_fig.add_axes([ax.get_position().x1+0.05, ax.get_position().y0, 0.02, ax.get_position().height])
#         cax0 = group_fig.add_axes([ax.get_position().x1 + 0.04, ax.get_position().y0, 0.01, 0.755])
#         cbar0 = group_fig.colorbar(pm0, cax = cax0, ticks = ticks_tpw)
#         cbar0.set_label('TPW [mm]')
#         cbar0.ax.set_yticklabels(list(map(str, list(ticks_tpw))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
#         cbar0.ax.yaxis.set_ticks_position('left')
#         cbar0.ax.yaxis.set_label_position('left')

#         #IMERG colorbar
#         ticks_imerg = np.array([0.1, 1, 5, 10, 20, 40])
#         cax1 = group_fig.add_axes([ax.get_position().x1 + 0.04, ax.get_position().y0, 0.01, 0.755])
#         cbar1 = group_fig.colorbar(pm1, cax = cax1, ticks = ticks_imerg)
#         cbar1.set_label('IMERG [mm hr$\\bf{^{-1}}$]')
#         cbar1.ax.set_yticklabels(list(map(str, list(ticks_imerg))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
#         cbar1.ax.yaxis.set_ticks_position('right')
#         cbar1.ax.yaxis.set_label_position('right')
                
#     ds_tpw.close()
#     ds_imerg.close()
#     ds.close()
    
#     print (file_date + ' convergence plots complete!\n')

# #plt.tight_layout()
# #plt.subplots_adjust(wspace = 0.3)

# #save the figure
# plt.savefig('/Users/brodenkirch/Desktop/Figure11_updated.png', bbox_inches = 'tight')
# #plt.show()  #plt.show() must come after plt.savefig() in order for the image to save properly
# #plt.clf()   #supposedly speeds things up? According to: https://www.youtube.com/watch?v=jGVIZbi9uMY
# plt.close()

# ##decrease file size of the image by 66% without noticeable image effects (if using Matplotlib)
# ##(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
# im = Image.open('/Users/brodenkirch/Desktop/Figure11_updated.png')

# try:
#     im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
# except:
#     #use this for older version of PIL/Pillow if the above line doesn't work, 
#     #though this line will have isolated, extremely minor image effects due to 
#     #only using 256 colors instead of the 3-element RGB scale
#     im2 = im.convert('P')

# im2.save('/Users/brodenkirch/Desktop/Figure11_updated.png')
# im.close()
# im2.close()

# print ('Done!')


#### Figure 15 for AGU Paper (2023) (merge the two plots together in PowerPoint)

In [ ]:
# #FIGURE 15a,c FOR AGU PAPER (2023)

# case_dict_agu = {0: ['20170611', 'Case 13', 17, [-95, -80, 20, 28]], 
#                  1: ['20210824', 'Case 16', 16, [-75, -60, 10, 18]]}  #flight track lat/lon extent [West,East,South,North]
# conv_pres = 975  #hPa; pressure level to plot ERA5 convergence for

# drop_metric_cases1316_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_OnlyCases13and16.csv')
# df_drop_cases1316 = pd.read_csv(drop_metric_cases1316_filepath)

# #set some baseline plot displays

# #matplotlib.rcParams['axes.facecolor'] = [0.9,0.9,0.9]
# matplotlib.rcParams['axes.labelsize'] = 16
# matplotlib.rcParams['axes.titlesize'] = 19
# matplotlib.rcParams['axes.labelweight'] = 'bold'
# matplotlib.rcParams['axes.titleweight'] = 'bold'
# matplotlib.rcParams['xtick.labelsize'] = 16
# matplotlib.rcParams['ytick.labelsize'] = 16
# matplotlib.rcParams['legend.fontsize'] = 16
# #matplotlib.rcParams['legend.facecolor'] = 'w'
# #matplotlib.rcParams['axes.facecolor'] = 'w'
# matplotlib.rcParams['font.family'] = 'arial'
# matplotlib.rcParams['hatch.linewidth'] = 0.3

# data_proj = ccrs.PlateCarree()
# group_fig = plt.figure(figsize = (30,15))

# for key, case_info in case_dict_agu.items():
    
#     file_date = case_info[0]
#     case_num = case_info[1]   
#     hr = case_info[2]
#     campaign_extent = case_info[3]

#     print (file_date + ' convergence plots in progress...')
    
#     ###get locations of the dropsonde/Navigation/ERA5 folder and read the appropriate files in
#     day_folder = os.path.join(os.getcwd(), file_date)

#     #dropsonde data
#     drop_csv_path = os.path.join(day_folder, 'final_dropsonde_' + file_date + '.csv')
#     drop_csv = pd.read_csv(drop_csv_path)

#     if file_date[:4] == '2017':
#         campaign = 'CPEX'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
#     elif file_date[:4] == '2021':
#         campaign = 'CPEXAW'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
#     elif file_date[:4] == '2022':
#         campaign = 'CPEXCV'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')
#     else:
#         pass

#     #ERA5 data
#     era5_folder = os.path.join(os.getcwd(), 'ERA5_Reanalysis_Data')
#     era5_path = os.path.join(era5_folder, campaign + '_ERA5_Reanalysis_Hourly_Pressure.nc')
#     ds = xr.open_dataset(era5_path)

#     #Navigation data
#     nav_folder = os.path.join(day_folder, 'Nav_files')

#     for x in os.listdir(nav_folder):
#         if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
#             os.remove(os.path.join(nav_folder, x))

#     nav_ict_path = os.path.join(nav_folder, os.listdir(nav_folder)[0])  #only one nav file per flight

#     if campaign != 'CPEX':  #campaign either CPEXAW or CPEXCV

#         nav_ict = icartt.Dataset(nav_ict_path)    #open the ict file with icartt
#         flight_lat = nav_ict.data["Latitude"]     #nav latitude, just a normal 1-D array
#         flight_lon = nav_ict.data["Longitude"]    #nav longitude, just a normal 1-D array

#     else:  #for CPEX navigation files, open the CSV file with pandas
#            #(Navigation files for CPEX (2017) are originally .kmz not .ict,
#            #so I converted them to CSV for free using https://www.gpsvisualizer.com/convert_input

#         nav_ict = pd.read_csv(nav_ict_path)       #open the ict file with pandas instead
#         flight_lat = nav_ict["latitude"].values   #nav latitude, just a normal 1-D array
#         flight_lon = nav_ict["longitude"].values  #nav longitude, just a normal 1-D array    


#     ###calculate each near-storm dropsonde's mean lat/lon and add the sonde's time and mean lat/lon to a list to be plotted

#     drop_coords_and_time = []   #format: longitude, latitude, time (HHSS)

#     df_drop = pd.read_csv(drop_metric_filepath)
#     df_drop_use = df_drop[df_drop['Date'] == int(file_date)].copy()

#     for x in range(len(df_drop_use)):
#         date = str(df_drop_use['Date'].iloc[x])
#         time = str(df_drop_use['Time'].iloc[x]).zfill(6)
#         sonde_datetime = date[:4] + '-' + date[4:6] + '-' + date[6:] + ' ' + time[:2] + ':' + time[2:4] + ':' + time[4:]

#         drop_csv_use = drop_csv[drop_csv['Time [UTC]'] == sonde_datetime].copy()
#         drop_mean_lon = drop_csv_use['Longitude [deg]'].mean()
#         drop_mean_lat = drop_csv_use['Latitude [deg]'].mean()

#         sonde_info = [drop_mean_lon, drop_mean_lat, time[:4], date, time]
#         drop_coords_and_time.append(sonde_info)
        
#     hr2 = str(hr).zfill(2)

#     #MIMIC TPW data
#         ##https://bin.ssec.wisc.edu/pub/mtpw2/data/
#     tpw_folder = os.path.join(day_folder, 'MIMIC_TPW_files')
#     tpw_path = os.path.join(tpw_folder, 'comp' + file_date + '.' + hr2 + '0000.nc')
#     ds_tpw = xr.open_dataset(tpw_path)

#     #GPM IMERG data (see IMERG.ipynb for more how to more generally download and plot IMERG data)
#         ##https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Read%20IMERG%20Data%20Using%20Python
#         ##https://disc.gsfc.nasa.gov/datasets?keywords=imerg&page=1
#         #0.1 x 0.1 gridded data, half-hourly means, using the half hour BEFORE the desired hour
#     imerg_folder = os.path.join(day_folder, 'IMERG_files')

#     for x in os.listdir(imerg_folder):
#         if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
#             os.remove(os.path.join(imerg_folder, x))

#         #minutes and seconds automatically revert to zero (hour = 0, seconds = 0) for '%Y%m%d%H'
#         elif (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(seconds = 1), '%H%M%S') in x) and (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(minutes = 30), '%H%M%S') in x):
#             imerg_file = x
#             break
#         else:
#             imerg_file = 'Could not find the desired IMERG file'

#     #confirm that the IMERG file is from the correct day (if hr2 == '00', then this will be the previous day)
#     assert (file_date in imerg_file) or (hr2 == '00'), 'IMERG file not from the correct day'

#     imerg_path = os.path.join(imerg_folder, imerg_file)
#     ds_imerg = h5py.File(imerg_path, 'r')

#     imerg_lons = ds_imerg['Grid/lon'][:]   #Longitude Shape: (3600,)
#     imerg_lats = ds_imerg['Grid/lat'][:]   #Latitude Shape: (1800,)
#     imerg_lons, imerg_lats = np.meshgrid(imerg_lons, imerg_lats)  #Long and lat grid shape: (1800, 3600) 

#     imerg_precip = ds_imerg['Grid/precipitation'][0][:][:]  #Original Precip Shape: (1, 3600, 1800) = (time, lon, lat)
#     imerg_precip = np.transpose(imerg_precip)               #New Precip Shape after transpose: (1800, 3600)

#     #mask blank data
#     imerg_precip_masked = np.ma.masked_where(imerg_precip < 0, imerg_precip)  #masks blank and bad data first (if blank data is -999 instead of NaN)
#     imerg_precip_masked = np.ma.masked_where(np.isnan(imerg_precip_masked), imerg_precip_masked)  #masks NaN values (not masked in previous line)

#     #creat the plot
#     ax = group_fig.add_subplot(2, 1, key+1, projection = data_proj)

#     conv = ds.d.sel(time = file_date).sel(level = conv_pres) * -1   #convergence of the wind (1/s)
#     conv = conv.sel(time = conv.time.dt.hour.isin(hr)) * 10**5      #convergence of the wind (times 10**5 1/s)

#     ##Smoothing (source: Hannah Zanowski) --> not recommended, see top of document
#         ##Metpy smooth_n_point (data to be smoothed, number of points to use in smoothing (5 to 9 are valid), and number of times the smoother is applied)
#             ##see https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.smooth_n_point.html for more info
#     #conv_smoothed = mpcalc.smooth_n_point(conv,9,1)

#     ax.set_title('%s MIMIC TPW, GPM IMERG, and ERA5 %i hPa Convergence (%s UTC)' % (case_num, conv_pres, hr2))
#     ax.set_extent(campaign_extent, ccrs.PlateCarree()) #lat/lon bounds are [West,East,South,North]

#     # Add land, coastlines, and borders
#     #ax.add_feature(cfeature.LAND, facecolor='0.8')
#     ax.coastlines(ls = '-', linewidth = 0.5, color = 'gray')

#     #plot MIMIC TPW
#     tpw_levels = np.arange(0, 70.5, 2)
#     pm0 = ax.contourf(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, levels = tpw_levels,
#                       extend = 'max', cmap = cm.jet, transform = data_proj)
# #             pm0 = ax.pcolormesh(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, vmin = 0, vmax = 70,
# #                                 cmap = cm.jet, transform = data_proj, zorder = 0)

#     #plot IMERG Rain Rate
#     pm1 = ax.contourf(imerg_lons, imerg_lats, imerg_precip_masked, 
#                       levels = np.logspace(np.log10(0.1), np.log10(40), num = len(tpw_levels)), 
#                       norm = 'log', extend = 'max', 
#                       cmap = cm.jet, transform = data_proj, zorder = 1)
# #             pm1 = ax.pcolormesh(imerg_lons, imerg_lats, imerg_precip_masked, 
# #                                 norm = mplc.LogNorm(vmin = 0.1, vmax = 40), 
# #                                 cmap = cm.jet, transform = data_proj, zorder = 1)

#     #gridlines
#     gl = ax.gridlines(crs = ccrs.PlateCarree(), draw_labels = True, linewidth = 0.5, 
#                       color = 'gray', alpha = 0.5, linestyle = '--', zorder = 2)
#     gl.top_labels = False
#     gl.right_labels = False
#     gl.xlabel_style = {'size':16, 'color':'black'}
#     gl.ylabel_style = {'size':16, 'color':'black'}
    
#     level_array = np.arange(1, 201.1, 2)
#     linewidth_array = np.full_like(level_array, 0.6)
#     #linewidth_array[3:] = 2.5   #values 7 x 10^-5 s^-1 and greater will have thicker contours
#     linewidth_array[2:] = 2.5   #values 5 x 10^-5 s^-1 and greater will have thicker contours

#     #plot ERA5 convergence
#     ax.contour(ds.longitude, ds.latitude, conv[0].values, levels = level_array,
#                colors = 'k', linewidths = linewidth_array, transform = data_proj, zorder = 3)            

#     #plot flight track
#     ax.plot(flight_lon, flight_lat, color = 'darkmagenta', linewidth = 2.5, zorder = 4)

#     #plot near-storm dropsonde locations for the given flight
#         #NOTE: Dropsondes with no wind data don't have GPS data either (5 of them total)
#     for sonde in drop_coords_and_time:
#         #ax.scatter(sonde[0], sonde[1], marker = f'${sonde[2]}$', color = 'b', s = 300)
#         df_drop_cases1316_use = df_drop_cases1316[(df_drop_cases1316['Date'] == int(sonde[3])) & (df_drop_cases1316['Time'] == int(sonde[4]))].copy()
        
#         #skip plotting dropsondes from the given flight that aren't associated with Case 13 nor Case 16
#         if len(df_drop_cases1316_use) == 0:
#             continue

#         if df_drop_cases1316_use['Beyond TPW Gradient'].iloc[0] == 'Yes':
#             ax.scatter(sonde[0], sonde[1], marker = '^', color = 'k', zorder = 5, s = 250)
#         else:
#             ax.scatter(sonde[0], sonde[1], marker = '*', color = 'k', zorder = 5, s = 400)

#     #same as above, but labeling the dropsondes by the order that they appear in the
#         #Dropsonde_Metric_Calculations.csv, NOT IN CHRONOLOGICAL ORDER!!!
#     #for z, sonde in enumerate(drop_coords_and_time):
#         #ax.scatter(sonde[0], sonde[1], marker = f'${z + 1}$', color = 'b', s = 120, zorder = 5)

#     #plotting the colorbars
#     #cbar0 = group_fig.colorbar(pm0, ax = ax, orientation = 'vertical', shrink = 0.75, pad = 0.25)
#     #cbar0.set_label('TPW [mm]')
#     #cbar0.ax.yaxis.set_ticks_position('left')
#     #cbar0.ax.yaxis.set_label_position('left')

#     #this works with GeoAxes (i.e., Cartopy's map projections)
#     if key == len(case_dict_agu) - 1:

#         #MIMIC TPW colorbar
#         ticks_tpw = np.arange(0, 70.5, 10)
#         #cax = group_fig.add_axes([ax.get_position().x1+0.05, ax.get_position().y0, 0.02, ax.get_position().height])
#         cax0 = group_fig.add_axes([ax.get_position().x1 + 0.04, ax.get_position().y0, 0.01, 0.755])
#         cbar0 = group_fig.colorbar(pm0, cax = cax0, ticks = ticks_tpw)
#         cbar0.set_label('TPW [mm]')
#         cbar0.ax.set_yticklabels(list(map(str, list(ticks_tpw))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
#         cbar0.ax.yaxis.set_ticks_position('left')
#         cbar0.ax.yaxis.set_label_position('left')

#         #IMERG colorbar
#         ticks_imerg = np.array([0.1, 1, 5, 10, 20, 40])
#         cax1 = group_fig.add_axes([ax.get_position().x1 + 0.04, ax.get_position().y0, 0.01, 0.755])
#         cbar1 = group_fig.colorbar(pm1, cax = cax1, ticks = ticks_imerg)
#         cbar1.set_label('IMERG [mm hr$\\bf{^{-1}}$]')
#         cbar1.ax.set_yticklabels(list(map(str, list(ticks_imerg))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
#         cbar1.ax.yaxis.set_ticks_position('right')
#         cbar1.ax.yaxis.set_label_position('right')
                
#     ds_tpw.close()
#     ds_imerg.close()
#     ds.close()
    
#     print (file_date + ' convergence plots complete!\n')

# #plt.tight_layout()
# #plt.subplots_adjust(wspace = 0.3)

# #save the figure
# plt.savefig('/Users/brodenkirch/Desktop/Figure15ac_updated.png', bbox_inches = 'tight')
# #plt.show()  #plt.show() must come after plt.savefig() in order for the image to save properly
# #plt.clf()   #supposedly speeds things up? According to: https://www.youtube.com/watch?v=jGVIZbi9uMY
# plt.close()

# ##decrease file size of the image by 66% without noticeable image effects (if using Matplotlib)
# ##(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
# im = Image.open('/Users/brodenkirch/Desktop/Figure15ac_updated.png')

# try:
#     im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
# except:
#     #use this for older version of PIL/Pillow if the above line doesn't work, 
#     #though this line will have isolated, extremely minor image effects due to 
#     #only using 256 colors instead of the 3-element RGB scale
#     im2 = im.convert('P')

# im2.save('/Users/brodenkirch/Desktop/Figure15ac_updated.png')
# im.close()
# im2.close()

# print ('Done!')


In [ ]:
# #FIGURE 15b,d FOR AGU PAPER (2023)

# case_dict_agu = {0: ['20170611', 'Case 13', 19, [-95, -80, 20, 28]], 
#                  1: ['20210824', 'Case 16', 18, [-75, -60, 10, 18]]}  #flight track lat/lon extent [West,East,South,North]
# conv_pres = 975  #hPa; pressure level to plot ERA5 convergence for

# drop_metric_cases1316_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_OnlyCases13and16.csv')
# df_drop_cases1316 = pd.read_csv(drop_metric_cases1316_filepath)

# #set some baseline plot displays

# #matplotlib.rcParams['axes.facecolor'] = [0.9,0.9,0.9]
# matplotlib.rcParams['axes.labelsize'] = 16
# matplotlib.rcParams['axes.titlesize'] = 19
# matplotlib.rcParams['axes.labelweight'] = 'bold'
# matplotlib.rcParams['axes.titleweight'] = 'bold'
# matplotlib.rcParams['xtick.labelsize'] = 16
# matplotlib.rcParams['ytick.labelsize'] = 16
# matplotlib.rcParams['legend.fontsize'] = 16
# #matplotlib.rcParams['legend.facecolor'] = 'w'
# #matplotlib.rcParams['axes.facecolor'] = 'w'
# matplotlib.rcParams['font.family'] = 'arial'
# matplotlib.rcParams['hatch.linewidth'] = 0.3

# data_proj = ccrs.PlateCarree()
# group_fig = plt.figure(figsize = (30,15))

# for key, case_info in case_dict_agu.items():
    
#     file_date = case_info[0]
#     case_num = case_info[1]   
#     hr = case_info[2]
#     campaign_extent = case_info[3]

#     print (file_date + ' convergence plots in progress...')
    
#     ###get locations of the dropsonde/Navigation/ERA5 folder and read the appropriate files in
#     day_folder = os.path.join(os.getcwd(), file_date)

#     #dropsonde data
#     drop_csv_path = os.path.join(day_folder, 'final_dropsonde_' + file_date + '.csv')
#     drop_csv = pd.read_csv(drop_csv_path)

#     if file_date[:4] == '2017':
#         campaign = 'CPEX'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
#     elif file_date[:4] == '2021':
#         campaign = 'CPEXAW'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
#     elif file_date[:4] == '2022':
#         campaign = 'CPEXCV'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')
#     else:
#         pass

#     #ERA5 data
#     era5_folder = os.path.join(os.getcwd(), 'ERA5_Reanalysis_Data')
#     era5_path = os.path.join(era5_folder, campaign + '_ERA5_Reanalysis_Hourly_Pressure.nc')
#     ds = xr.open_dataset(era5_path)

#     #Navigation data
#     nav_folder = os.path.join(day_folder, 'Nav_files')

#     for x in os.listdir(nav_folder):
#         if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
#             os.remove(os.path.join(nav_folder, x))

#     nav_ict_path = os.path.join(nav_folder, os.listdir(nav_folder)[0])  #only one nav file per flight

#     if campaign != 'CPEX':  #campaign either CPEXAW or CPEXCV

#         nav_ict = icartt.Dataset(nav_ict_path)    #open the ict file with icartt
#         flight_lat = nav_ict.data["Latitude"]     #nav latitude, just a normal 1-D array
#         flight_lon = nav_ict.data["Longitude"]    #nav longitude, just a normal 1-D array

#     else:  #for CPEX navigation files, open the CSV file with pandas
#            #(Navigation files for CPEX (2017) are originally .kmz not .ict,
#            #so I converted them to CSV for free using https://www.gpsvisualizer.com/convert_input

#         nav_ict = pd.read_csv(nav_ict_path)       #open the ict file with pandas instead
#         flight_lat = nav_ict["latitude"].values   #nav latitude, just a normal 1-D array
#         flight_lon = nav_ict["longitude"].values  #nav longitude, just a normal 1-D array    


#     ###calculate each near-storm dropsonde's mean lat/lon and add the sonde's time and mean lat/lon to a list to be plotted

#     drop_coords_and_time = []   #format: longitude, latitude, time (HHSS)

#     df_drop = pd.read_csv(drop_metric_filepath)
#     df_drop_use = df_drop[df_drop['Date'] == int(file_date)].copy()

#     for x in range(len(df_drop_use)):
#         date = str(df_drop_use['Date'].iloc[x])
#         time = str(df_drop_use['Time'].iloc[x]).zfill(6)
#         sonde_datetime = date[:4] + '-' + date[4:6] + '-' + date[6:] + ' ' + time[:2] + ':' + time[2:4] + ':' + time[4:]

#         drop_csv_use = drop_csv[drop_csv['Time [UTC]'] == sonde_datetime].copy()
#         drop_mean_lon = drop_csv_use['Longitude [deg]'].mean()
#         drop_mean_lat = drop_csv_use['Latitude [deg]'].mean()

#         sonde_info = [drop_mean_lon, drop_mean_lat, time[:4], date, time]
#         drop_coords_and_time.append(sonde_info)
        
#     hr2 = str(hr).zfill(2)

#     #MIMIC TPW data
#         ##https://bin.ssec.wisc.edu/pub/mtpw2/data/
#     tpw_folder = os.path.join(day_folder, 'MIMIC_TPW_files')
#     tpw_path = os.path.join(tpw_folder, 'comp' + file_date + '.' + hr2 + '0000.nc')
#     ds_tpw = xr.open_dataset(tpw_path)

#     #GPM IMERG data (see IMERG.ipynb for more how to more generally download and plot IMERG data)
#         ##https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Read%20IMERG%20Data%20Using%20Python
#         ##https://disc.gsfc.nasa.gov/datasets?keywords=imerg&page=1
#         #0.1 x 0.1 gridded data, half-hourly means, using the half hour BEFORE the desired hour
#     imerg_folder = os.path.join(day_folder, 'IMERG_files')

#     for x in os.listdir(imerg_folder):
#         if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
#             os.remove(os.path.join(imerg_folder, x))

#         #minutes and seconds automatically revert to zero (hour = 0, seconds = 0) for '%Y%m%d%H'
#         elif (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(seconds = 1), '%H%M%S') in x) and (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(minutes = 30), '%H%M%S') in x):
#             imerg_file = x
#             break
#         else:
#             imerg_file = 'Could not find the desired IMERG file'

#     #confirm that the IMERG file is from the correct day (if hr2 == '00', then this will be the previous day)
#     assert (file_date in imerg_file) or (hr2 == '00'), 'IMERG file not from the correct day'

#     imerg_path = os.path.join(imerg_folder, imerg_file)
#     ds_imerg = h5py.File(imerg_path, 'r')

#     imerg_lons = ds_imerg['Grid/lon'][:]   #Longitude Shape: (3600,)
#     imerg_lats = ds_imerg['Grid/lat'][:]   #Latitude Shape: (1800,)
#     imerg_lons, imerg_lats = np.meshgrid(imerg_lons, imerg_lats)  #Long and lat grid shape: (1800, 3600) 

#     imerg_precip = ds_imerg['Grid/precipitation'][0][:][:]  #Original Precip Shape: (1, 3600, 1800) = (time, lon, lat)
#     imerg_precip = np.transpose(imerg_precip)               #New Precip Shape after transpose: (1800, 3600)

#     #mask blank data
#     imerg_precip_masked = np.ma.masked_where(imerg_precip < 0, imerg_precip)  #masks blank and bad data first (if blank data is -999 instead of NaN)
#     imerg_precip_masked = np.ma.masked_where(np.isnan(imerg_precip_masked), imerg_precip_masked)  #masks NaN values (not masked in previous line)

#     #creat the plot
#     ax = group_fig.add_subplot(2, 1, key+1, projection = data_proj)

#     conv = ds.d.sel(time = file_date).sel(level = conv_pres) * -1   #convergence of the wind (1/s)
#     conv = conv.sel(time = conv.time.dt.hour.isin(hr)) * 10**5      #convergence of the wind (times 10**5 1/s)

#     ##Smoothing (source: Hannah Zanowski) --> not recommended, see top of document
#         ##Metpy smooth_n_point (data to be smoothed, number of points to use in smoothing (5 to 9 are valid), and number of times the smoother is applied)
#             ##see https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.smooth_n_point.html for more info
#     #conv_smoothed = mpcalc.smooth_n_point(conv,9,1)

#     ax.set_title('%s MIMIC TPW, GPM IMERG, and ERA5 %i hPa Convergence (%s UTC)' % (case_num, conv_pres, hr2))
#     ax.set_extent(campaign_extent, ccrs.PlateCarree()) #lat/lon bounds are [West,East,South,North]

#     # Add land, coastlines, and borders
#     #ax.add_feature(cfeature.LAND, facecolor='0.8')
#     ax.coastlines(ls = '-', linewidth = 0.5, color = 'gray')

#     #plot MIMIC TPW
#     tpw_levels = np.arange(0, 70.5, 2)
#     pm0 = ax.contourf(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, levels = tpw_levels,
#                       extend = 'max', cmap = cm.jet, transform = data_proj)
# #             pm0 = ax.pcolormesh(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, vmin = 0, vmax = 70,
# #                                 cmap = cm.jet, transform = data_proj, zorder = 0)

#     #plot IMERG Rain Rate
#     pm1 = ax.contourf(imerg_lons, imerg_lats, imerg_precip_masked, 
#                       levels = np.logspace(np.log10(0.1), np.log10(40), num = len(tpw_levels)), 
#                       norm = 'log', extend = 'max', 
#                       cmap = cm.jet, transform = data_proj, zorder = 1)
# #             pm1 = ax.pcolormesh(imerg_lons, imerg_lats, imerg_precip_masked, 
# #                                 norm = mplc.LogNorm(vmin = 0.1, vmax = 40), 
# #                                 cmap = cm.jet, transform = data_proj, zorder = 1)

#     #gridlines
#     gl = ax.gridlines(crs = ccrs.PlateCarree(), draw_labels = True, linewidth = 0.5, 
#                       color = 'gray', alpha = 0.5, linestyle = '--', zorder = 2)
#     gl.top_labels = False
#     gl.right_labels = False
#     gl.xlabel_style = {'size':16, 'color':'black'}
#     gl.ylabel_style = {'size':16, 'color':'black'}
    
#     level_array = np.arange(1, 201.1, 2)
#     linewidth_array = np.full_like(level_array, 0.6)
#     #linewidth_array[3:] = 2.5   #values 7 x 10^-5 s^-1 and greater will have thicker contours
#     linewidth_array[2:] = 2.5   #values 5 x 10^-5 s^-1 and greater will have thicker contours

#     #plot ERA5 convergence
#     ax.contour(ds.longitude, ds.latitude, conv[0].values, levels = level_array,
#                colors = 'k', linewidths = linewidth_array, transform = data_proj, zorder = 3)            

#     #plot flight track
#     ax.plot(flight_lon, flight_lat, color = 'darkmagenta', linewidth = 2.5, zorder = 4)

#     #plot near-storm dropsonde locations for the given flight
#         #NOTE: Dropsondes with no wind data don't have GPS data either (5 of them total)
#     for sonde in drop_coords_and_time:
#         #ax.scatter(sonde[0], sonde[1], marker = f'${sonde[2]}$', color = 'b', s = 300)
#         df_drop_cases1316_use = df_drop_cases1316[(df_drop_cases1316['Date'] == int(sonde[3])) & (df_drop_cases1316['Time'] == int(sonde[4]))].copy()
        
#         #skip plotting dropsondes from the given flight that aren't associated with Case 13 nor Case 16
#         if len(df_drop_cases1316_use) == 0:
#             continue

#         if df_drop_cases1316_use['Beyond TPW Gradient'].iloc[0] == 'Yes':
#             ax.scatter(sonde[0], sonde[1], marker = '^', color = 'k', zorder = 5, s = 250)
#         else:
#             ax.scatter(sonde[0], sonde[1], marker = '*', color = 'k', zorder = 5, s = 400)

#     #same as above, but labeling the dropsondes by the order that they appear in the
#         #Dropsonde_Metric_Calculations.csv, NOT IN CHRONOLOGICAL ORDER!!!
#     #for z, sonde in enumerate(drop_coords_and_time):
#         #ax.scatter(sonde[0], sonde[1], marker = f'${z + 1}$', color = 'b', s = 120, zorder = 5)

#     #plotting the colorbars
#     #cbar0 = group_fig.colorbar(pm0, ax = ax, orientation = 'vertical', shrink = 0.75, pad = 0.25)
#     #cbar0.set_label('TPW [mm]')
#     #cbar0.ax.yaxis.set_ticks_position('left')
#     #cbar0.ax.yaxis.set_label_position('left')

#     #this works with GeoAxes (i.e., Cartopy's map projections)
#     if key == len(case_dict_agu) - 1:

#         #MIMIC TPW colorbar
#         ticks_tpw = np.arange(0, 70.5, 10)
#         #cax = group_fig.add_axes([ax.get_position().x1+0.05, ax.get_position().y0, 0.02, ax.get_position().height])
#         cax0 = group_fig.add_axes([ax.get_position().x1 + 0.04, ax.get_position().y0, 0.01, 0.755])
#         cbar0 = group_fig.colorbar(pm0, cax = cax0, ticks = ticks_tpw)
#         cbar0.set_label('TPW [mm]')
#         cbar0.ax.set_yticklabels(list(map(str, list(ticks_tpw))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
#         cbar0.ax.yaxis.set_ticks_position('left')
#         cbar0.ax.yaxis.set_label_position('left')

#         #IMERG colorbar
#         ticks_imerg = np.array([0.1, 1, 5, 10, 20, 40])
#         cax1 = group_fig.add_axes([ax.get_position().x1 + 0.04, ax.get_position().y0, 0.01, 0.755])
#         cbar1 = group_fig.colorbar(pm1, cax = cax1, ticks = ticks_imerg)
#         cbar1.set_label('IMERG [mm hr$\\bf{^{-1}}$]')
#         cbar1.ax.set_yticklabels(list(map(str, list(ticks_imerg))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
#         cbar1.ax.yaxis.set_ticks_position('right')
#         cbar1.ax.yaxis.set_label_position('right')
                
#     ds_tpw.close()
#     ds_imerg.close()
#     ds.close()
    
#     print (file_date + ' convergence plots complete!\n')

# #plt.tight_layout()
# #plt.subplots_adjust(wspace = 0.3)

# #save the figure
# plt.savefig('/Users/brodenkirch/Desktop/Figure15bd_updated.png', bbox_inches = 'tight')
# #plt.show()  #plt.show() must come after plt.savefig() in order for the image to save properly
# #plt.clf()   #supposedly speeds things up? According to: https://www.youtube.com/watch?v=jGVIZbi9uMY
# plt.close()

# ##decrease file size of the image by 66% without noticeable image effects (if using Matplotlib)
# ##(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
# im = Image.open('/Users/brodenkirch/Desktop/Figure15bd_updated.png')

# try:
#     im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
# except:
#     #use this for older version of PIL/Pillow if the above line doesn't work, 
#     #though this line will have isolated, extremely minor image effects due to 
#     #only using 256 colors instead of the 3-element RGB scale
#     im2 = im.convert('P')

# im2.save('/Users/brodenkirch/Desktop/Figure15bd_updated.png')
# im.close()
# im2.close()

# print ('Done!')


#### Figure 2 for AGU Paper (2023)

In [ ]:
# #FIGURE 2 FOR AGU PAPER (2023)

# case_dict_agu = {0: ['20170610', 'an Isolated', 20, [-85, -60, 19, 31]], 
#                  1: ['20170601', 'an Organized', 19, [-98, -73, 19, 31]], 
#                  2: ['20170602', 'a Scattered', 22, [-98, -73, 19, 31]]}  #flight track lat/lon extent [West,East,South,North]

# #set some baseline plot displays

# #matplotlib.rcParams['axes.facecolor'] = [0.9,0.9,0.9]
# matplotlib.rcParams['axes.labelsize'] = 16
# matplotlib.rcParams['axes.titlesize'] = 18
# matplotlib.rcParams['axes.labelweight'] = 'bold'
# matplotlib.rcParams['axes.titleweight'] = 'bold'
# matplotlib.rcParams['xtick.labelsize'] = 16
# matplotlib.rcParams['ytick.labelsize'] = 16
# matplotlib.rcParams['legend.fontsize'] = 16
# #matplotlib.rcParams['legend.facecolor'] = 'w'
# #matplotlib.rcParams['axes.facecolor'] = 'w'
# matplotlib.rcParams['font.family'] = 'arial'
# matplotlib.rcParams['hatch.linewidth'] = 0.3

# data_proj = ccrs.PlateCarree()
# group_fig = plt.figure(figsize = (48,18))

# for key, case_info in case_dict_agu.items():
    
#     file_date = case_info[0]
#     conv_type = case_info[1]   
#     hr = case_info[2]
#     campaign_extent = case_info[3]

#     print (file_date + ' IMERG plots in progress...')
    
#     ###get locations of the dropsonde/Navigation/ERA5 folder and read the appropriate files in
#     day_folder = os.path.join(os.getcwd(), file_date)

#     #dropsonde data
#     drop_csv_path = os.path.join(day_folder, 'final_dropsonde_' + file_date + '.csv')
#     drop_csv = pd.read_csv(drop_csv_path)

#     if file_date[:4] == '2017':
#         campaign = 'CPEX'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
#     elif file_date[:4] == '2021':
#         campaign = 'CPEXAW'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
#     elif file_date[:4] == '2022':
#         campaign = 'CPEXCV'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')
#     else:
#         pass

#     #Navigation data
#     nav_folder = os.path.join(day_folder, 'Nav_files')

#     for x in os.listdir(nav_folder):
#         if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
#             os.remove(os.path.join(nav_folder, x))

#     nav_ict_path = os.path.join(nav_folder, os.listdir(nav_folder)[0])  #only one nav file per flight

#     if campaign != 'CPEX':  #campaign either CPEXAW or CPEXCV

#         nav_ict = icartt.Dataset(nav_ict_path)    #open the ict file with icartt
#         flight_lat = nav_ict.data["Latitude"]     #nav latitude, just a normal 1-D array
#         flight_lon = nav_ict.data["Longitude"]    #nav longitude, just a normal 1-D array

#     else:  #for CPEX navigation files, open the CSV file with pandas
#            #(Navigation files for CPEX (2017) are originally .kmz not .ict,
#            #so I converted them to CSV for free using https://www.gpsvisualizer.com/convert_input

#         nav_ict = pd.read_csv(nav_ict_path)       #open the ict file with pandas instead
#         flight_lat = nav_ict["latitude"].values   #nav latitude, just a normal 1-D array
#         flight_lon = nav_ict["longitude"].values  #nav longitude, just a normal 1-D array    


#     ###calculate each near-storm dropsonde's mean lat/lon and add the sonde's time and mean lat/lon to a list to be plotted

#     drop_coords_and_time = []   #format: longitude, latitude, time (HHSS)

#     df_drop = pd.read_csv(drop_metric_filepath)
#     df_drop_use = df_drop[df_drop['Date'] == int(file_date)].copy()

#     for x in range(len(df_drop_use)):
#         date = str(df_drop_use['Date'].iloc[x])
#         time = str(df_drop_use['Time'].iloc[x]).zfill(6)
#         sonde_datetime = date[:4] + '-' + date[4:6] + '-' + date[6:] + ' ' + time[:2] + ':' + time[2:4] + ':' + time[4:]

#         drop_csv_use = drop_csv[drop_csv['Time [UTC]'] == sonde_datetime].copy()
#         drop_mean_lon = drop_csv_use['Longitude [deg]'].mean()
#         drop_mean_lat = drop_csv_use['Latitude [deg]'].mean()

#         sonde_info = [drop_mean_lon, drop_mean_lat, time[:4]]
#         drop_coords_and_time.append(sonde_info)
        
#     hr2 = str(hr).zfill(2)

#     #GPM IMERG data (see IMERG.ipynb for more how to more generally download and plot IMERG data)
#        ##https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Read%20IMERG%20Data%20Using%20Python
#        ##https://disc.gsfc.nasa.gov/datasets?keywords=imerg&page=1
#        #0.1 x 0.1 gridded data, half-hourly means, using the half hour BEFORE the desired hour
#     imerg_folder = os.path.join(day_folder, 'IMERG_files')

#     for x in os.listdir(imerg_folder):
#         if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
#             os.remove(os.path.join(imerg_folder, x))

#         #minutes and seconds automatically revert to zero (hour = 0, seconds = 0) for '%Y%m%d%H'
#         elif (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(seconds = 1), '%H%M%S') in x) and (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(minutes = 30), '%H%M%S') in x):
#             imerg_file = x
#             break
#         else:
#             imerg_file = 'Could not find the desired IMERG file'

#     #confirm that the IMERG file is from the correct day (if hr2 == '00', then this will be the previous day)
#     assert (file_date in imerg_file) or (hr2 == '00'), 'IMERG file not from the correct day'

#     imerg_path = os.path.join(imerg_folder, imerg_file)
#     ds_imerg = h5py.File(imerg_path, 'r')

#     imerg_lons = ds_imerg['Grid/lon'][:]   #Longitude Shape: (3600,)
#     imerg_lats = ds_imerg['Grid/lat'][:]   #Latitude Shape: (1800,)
#     imerg_lons, imerg_lats = np.meshgrid(imerg_lons, imerg_lats)  #Long and lat grid shape: (1800, 3600) 

#     imerg_precip = ds_imerg['Grid/precipitation'][0][:][:]  #Original Precip Shape: (1, 3600, 1800) = (time, lon, lat)
#     imerg_precip = np.transpose(imerg_precip)               #New Precip Shape after transpose: (1800, 3600)

#     #mask blank data
#     imerg_precip_masked = np.ma.masked_where(imerg_precip < 0, imerg_precip)  #masks blank and bad data first (if blank data is -999 instead of NaN)
#     imerg_precip_masked = np.ma.masked_where(np.isnan(imerg_precip_masked), imerg_precip_masked)  #masks NaN values (not masked in previous line)

#     #creat the plot
#     ax = group_fig.add_subplot(3, 1, key+1, projection = data_proj)

#     ax.set_title('GPM IMERG for %s Convective Case' % (conv_type))
#     ax.set_extent(campaign_extent, ccrs.PlateCarree()) #lat/lon bounds are [West,East,South,North]

#     # Add land, coastlines, and borders
#     #ax.add_feature(cfeature.LAND, facecolor='0.8')
#     ax.coastlines(ls = '-', linewidth = 0.5, color = 'gray')

#     #plot IMERG Rain Rate
#     tpw_levels = np.arange(0, 70.5, 2)
#     pm1 = ax.contourf(imerg_lons, imerg_lats, imerg_precip_masked, 
#                       levels = np.logspace(np.log10(0.1), np.log10(40), num = len(tpw_levels)), 
#                       norm = 'log', extend = 'max', 
#                       cmap = cm.jet, transform = data_proj, zorder = 1)
# #             pm1 = ax.pcolormesh(imerg_lons, imerg_lats, imerg_precip_masked, 
# #                                 norm = mplc.LogNorm(vmin = 0.1, vmax = 40), 
# #                                 cmap = cm.jet, transform = data_proj, zorder = 1)

#     #gridlines
#     gl = ax.gridlines(crs = ccrs.PlateCarree(), draw_labels = True, linewidth = 0.5, 
#                       color = 'gray', alpha = 0.5, linestyle = '--', zorder = 2)
#     gl.top_labels = False
#     gl.right_labels = False
#     gl.xlabel_style = {'size':16, 'color':'black'}
#     gl.ylabel_style = {'size':16, 'color':'black'}            

#     #plot flight track
#     ax.plot(flight_lon, flight_lat, color = 'salmon', linewidth = 1.5, zorder = 4)

#     #plot near-storm dropsonde locations for the given flight
#         #NOTE: Dropsondes with no wind data don't have GPS data either (5 of them total)
#     for sonde in drop_coords_and_time:
#         #ax.scatter(sonde[0], sonde[1], marker = f'${sonde[2]}$', color = 'b', s = 300)
#         ax.scatter(sonde[0], sonde[1], marker = '*', color = 'k', zorder = 5)

#     #same as above, but labeling the dropsondes by the order that they appear in the
#         #Dropsonde_Metric_Calculations.csv, NOT IN CHRONOLOGICAL ORDER!!!
#     #for z, sonde in enumerate(drop_coords_and_time):
#         #ax.scatter(sonde[0], sonde[1], marker = f'${z + 1}$', color = 'b', s = 120, zorder = 5)

#     #plotting the colorbars
#     #cbar0 = group_fig.colorbar(pm0, ax = ax, orientation = 'vertical', shrink = 0.75, pad = 0.25)
#     #cbar0.set_label('IMERG [mm hr$\\bf{^{-1}}$]')
#     #cbar0.ax.yaxis.set_ticks_position('left')
#     #cbar0.ax.yaxis.set_label_position('left')

#     #this works with GeoAxes (i.e., Cartopy's map projections)
#     if key == len(case_dict_agu) - 1:

#         #IMERG colorbar
#         ticks_imerg = np.array([0.1, 1, 5, 10, 20, 40])
#         cax1 = group_fig.add_axes([ax.get_position().x1 + 0.01, ax.get_position().y0, 0.01, 0.755])
#         cbar1 = group_fig.colorbar(pm1, cax = cax1, ticks = ticks_imerg)
#         cbar1.set_label('IMERG [mm hr$\\bf{^{-1}}$]')
#         cbar1.ax.set_yticklabels(list(map(str, list(ticks_imerg))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
#         cbar1.ax.yaxis.set_ticks_position('right')
#         cbar1.ax.yaxis.set_label_position('right')
                
#     ds_imerg.close()
    
#     print (file_date + ' IMERG plots complete!\n')

# #plt.tight_layout()
# #plt.subplots_adjust(wspace = 0.3)

# #save the figure
# plt.savefig('/Users/brodenkirch/Desktop/Figure2_updated.png', bbox_inches = 'tight')
# #plt.show()  #plt.show() must come after plt.savefig() in order for the image to save properly
# #plt.clf()   #supposedly speeds things up? According to: https://www.youtube.com/watch?v=jGVIZbi9uMY
# plt.close()

# ##decrease file size of the image by 66% without noticeable image effects (if using Matplotlib)
# ##(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
# im = Image.open('/Users/brodenkirch/Desktop/Figure2_updated.png')

# try:
#     im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
# except:
#     #use this for older version of PIL/Pillow if the above line doesn't work, 
#     #though this line will have isolated, extremely minor image effects due to 
#     #only using 256 colors instead of the 3-element RGB scale
#     im2 = im.convert('P')

# im2.save('/Users/brodenkirch/Desktop/Figure2_updated.png')
# im.close()
# im2.close()

# print ('Done!')


### For CPEX-CV AGU Paper (Figure 2)

In [ ]:
# #adapted from CPEXCV_BAMS_Figures.ipynb (Figure 7)
# #plotting ERA5 reanalysis 850 hPa streamlines (and maybe RH), MIMIC TPW (if not ERA5 RH) and GPM IMERG

# # case_dict_agu = {0: ['20220909', '2022-09-09', 16, [-60, -10, 0, 30], 'Isolated'],
# #                 1: ['20220907', '2022-09-07', 15, [-60, -10, 0, 30], 'Organized']}  #flight track lat/lon extent [West,East,South,North]
# case_dict_agu = {0: ['20220909', '2022-09-09', 16, [-27.5, -12.5, 10, 20], 'Isolated'],
#                 1: ['20220907', '2022-09-07', 15, [-50, -20, 5, 25], 'Organized']}  #flight track lat/lon extent [West,East,South,North]

# stream_pres = 700  #hPa; pressure level to plot ERA5 streamlines for

# #set some baseline plot displays

# #matplotlib.rcParams['axes.facecolor'] = [0.9,0.9,0.9]
# matplotlib.rcParams['axes.labelsize'] = 40.5
# matplotlib.rcParams['axes.titlesize'] = 42.5
# matplotlib.rcParams['axes.labelweight'] = 'bold'
# matplotlib.rcParams['axes.titleweight'] = 'bold'
# matplotlib.rcParams['xtick.labelsize'] = 40.5
# matplotlib.rcParams['ytick.labelsize'] = 40.5
# matplotlib.rcParams['legend.fontsize'] = 40.5
# matplotlib.rcParams['legend.facecolor'] = 'w'
# matplotlib.rcParams['axes.facecolor'] = 'w'
# matplotlib.rcParams['font.family'] = 'arial'
# matplotlib.rcParams['hatch.linewidth'] = 0.3

# data_proj = ccrs.PlateCarree()
# group_fig = plt.figure(figsize = (24,34))

# for key, case_info in case_dict_agu.items():
    
#     file_date = case_info[0]
#     file_date_prior = str(int(case_info[0]) - 1)
#     case_num = case_info[1]   
#     hr = case_info[2]
#     campaign_extent = case_info[3]
#     convective_type = case_info[4]
    
#     ###get locations of the dropsonde/Navigation folder and read the appropriate files in
#     day_folder = os.path.join(os.getcwd(), file_date)

# #     #dropsonde data
# #     drop_csv_path = os.path.join(day_folder, 'final_dropsonde_' + file_date + '.csv')
# #     drop_csv = pd.read_csv(drop_csv_path)

#     if file_date[:4] == '2017':
#         campaign = 'CPEX'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
#     elif file_date[:4] == '2021':
#         campaign = 'CPEXAW'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
#     elif file_date[:4] == '2022':
#         campaign = 'CPEXCV'
#         drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')
#     else:
#         pass

#     #ERA5 data
#     era5_folder = os.path.join(os.getcwd(), 'ERA5_Reanalysis_Data')
#     era5_path = os.path.join(era5_folder, campaign + '_ERA5_Reanalysis_Hourly_Pressure.nc')
#     #era5_path = os.path.join(era5_folder, campaign + '_ERA5_Reanalysis_Hourly_Pressure_BAMS.nc')
#     ds = xr.open_dataset(era5_path)

#     #Navigation data
#     nav_folder = os.path.join(day_folder, 'Nav_files')
    
#     for x in os.listdir(nav_folder):
#         if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
#             os.remove(os.path.join(nav_folder, x))

#     nav_ict_path = os.path.join(nav_folder, os.listdir(nav_folder)[0])  #only one nav file per flight

#     if campaign != 'CPEX':  #campaign either CPEXAW or CPEXCV

#         nav_ict = icartt.Dataset(nav_ict_path)    #open the ict file with icartt
#         flight_lat = nav_ict.data["Latitude"]     #nav latitude, just a normal 1-D array
#         flight_lon = nav_ict.data["Longitude"]    #nav longitude, just a normal 1-D array

#     else:  #for CPEX navigation files, open the CSV file with pandas
#            #(Navigation files for CPEX (2017) are originally .kmz not .ict,
#            #so I converted them to CSV for free using https://www.gpsvisualizer.com/convert_input

#         nav_ict = pd.read_csv(nav_ict_path)       #open the ict file with pandas instead
#         flight_lat = nav_ict["latitude"].values   #nav latitude, just a normal 1-D array
#         flight_lon = nav_ict["longitude"].values  #nav longitude, just a normal 1-D array    


#     # ###calculate each dropsonde's mean lat/lon and add the sonde's time and mean lat/lon to a list to be plotted

#     # drop_coords_and_time = []   #format: longitude, latitude, time (HHSS)

#     # for sonde_datetime in drop_csv['Time [UTC]'].unique():
#     #     drop_csv_use = drop_csv[drop_csv['Time [UTC]'] == sonde_datetime].copy()
#     #     drop_mean_lon = drop_csv_use['Longitude [deg]'].mean()
#     #     drop_mean_lat = drop_csv_use['Latitude [deg]'].mean()

#     #     sonde_info = [drop_mean_lon, drop_mean_lat, sonde_datetime[-8:-3]]
#     #     drop_coords_and_time.append(sonde_info)

#     # ###calculate each NEAR-STORM dropsonde's mean lat/lon and add the sonde's time and mean lat/lon to a list to be plotted
#     # df_drop = pd.read_csv(drop_metric_filepath)
#     # df_drop_use = df_drop[df_drop['Date'] == int(file_date)].copy()

#     # for x in range(len(df_drop_use)):
#     #     date = str(df_drop_use['Date'].iloc[x])
#     #     time = str(df_drop_use['Time'].iloc[x]).zfill(6)
#     #     sonde_datetime = date[:4] + '-' + date[4:6] + '-' + date[6:] + ' ' + time[:2] + ':' + time[2:4] + ':' + time[4:]

#     #     drop_csv_use = drop_csv[drop_csv['Time [UTC]'] == sonde_datetime].copy()
#     #     drop_mean_lon = drop_csv_use['Longitude [deg]'].mean()
#     #     drop_mean_lat = drop_csv_use['Latitude [deg]'].mean()

#     #     sonde_info = [drop_mean_lon, drop_mean_lat, time[:4]]
#     #     drop_coords_and_time.append(sonde_info)
        
#     hr2 = str(hr).zfill(2)

#     #MIMIC TPW data
#         ##https://bin.ssec.wisc.edu/pub/mtpw2/data/
#     tpw_folder = os.path.join(day_folder, 'MIMIC_TPW_files')
#     tpw_path = os.path.join(tpw_folder, 'comp' + file_date + '.' + hr2 + '0000.nc')
#     ds_tpw = xr.open_dataset(tpw_path)

#     #GPM IMERG data (see IMERG.ipynb for more how to more generally download and plot IMERG data)
#         ##https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Read%20IMERG%20Data%20Using%20Python
#         ##https://disc.gsfc.nasa.gov/datasets?keywords=imerg&page=1
#         #0.1 x 0.1 gridded data, half-hourly means, using the half hour BEFORE the desired hour
#     imerg_folder = os.path.join(day_folder, 'IMERG_files')

#     for x in os.listdir(imerg_folder):
#         if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
#             os.remove(os.path.join(imerg_folder, x))

#         #minutes and seconds automatically revert to zero (hour = 0, seconds = 0) for '%Y%m%d%H'
#         elif (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(seconds = 1), '%H%M%S') in x) and (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(minutes = 30), '%H%M%S') in x):
#             imerg_file = x
#             break
#         else:
#             imerg_file = 'Could not find the desired IMERG file'

#     #confirm that the IMERG file is from the correct day (if hr2 == '00', then this will be the previous day)
#     assert (file_date in imerg_file) or (hr2 == '00'), 'IMERG file not from the correct day'

#     imerg_path = os.path.join(imerg_folder, imerg_file)
#     ds_imerg = h5py.File(imerg_path, 'r')

#     imerg_lons = ds_imerg['Grid/lon'][:]   #Longitude Shape: (3600,)
#     imerg_lats = ds_imerg['Grid/lat'][:]   #Latitude Shape: (1800,)
#     imerg_lons, imerg_lats = np.meshgrid(imerg_lons, imerg_lats)  #Long and lat grid shape: (1800, 3600) 

#     imerg_precip = ds_imerg['Grid/precipitation'][0][:][:]  #Original Precip Shape: (1, 3600, 1800) = (time, lon, lat)
#     imerg_precip = np.transpose(imerg_precip)               #New Precip Shape after transpose: (1800, 3600)

#     #mask blank data
#     imerg_precip_masked = np.ma.masked_where(imerg_precip < 0, imerg_precip)  #masks blank and bad data first (if blank data is -999 instead of NaN)
#     imerg_precip_masked = np.ma.masked_where(np.isnan(imerg_precip_masked), imerg_precip_masked)  #masks NaN values (not masked in previous line)

#     #creat the plot
#     ax = group_fig.add_subplot(2, 1, key+1, projection = data_proj)
    
#     uwnd = ds.u.sel(time = file_date).sel(level = stream_pres)  #zonal winds (m/s)
#     uwnd = uwnd.sel(time = uwnd.time.dt.hour.isin(hr))       #zonal winds (m/s)

#     vwnd = ds.v.sel(time = file_date).sel(level = stream_pres)  #meridional winds (m/s)
#     vwnd = vwnd.sel(time = vwnd.time.dt.hour.isin(hr))       #meridional winds (m/s)
    
#     rh = ds.r.sel(time = file_date).sel(level = stream_pres)  #meridional winds (m/s)
#     rh = rh.sel(time = rh.time.dt.hour.isin(hr))       #meridional winds (m/s)

#     ##Smoothing (source: Hannah Zanowski) --> not recommended, see top of document
#         ##Metpy smooth_n_point (data to be smoothed, number of points to use in smoothing (5 to 9 are valid), and number of times the smoother is applied)
#             ##see https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.smooth_n_point.html for more info
#     #uwnd_smoothed = mpcalc.smooth_n_point(uwnd,9,10)
#     #vwnd_smoothed = mpcalc.smooth_n_point(vwnd,9,10)

#     #ax.set_title('%s %s UTC MIMIC TPW, GPM IMERG, and ERA5 %i hPa Streamlines' % (case_num, hr2, stream_pres))
#     ax.set_title('GPM IMERG and MIMIC TPW for an %s Convective Case' % (convective_type))
#     #ax.set_title('%s %s UTC ERA5 %i hPa Relative Humidity, \nGPM IMERG, and ERA5 %i hPa Streamlines' % (case_num, hr2, stream_pres, stream_pres))
#     ax.set_extent(campaign_extent, ccrs.PlateCarree()) #lat/lon bounds are [West,East,South,North]

#     # Add land, coastlines, and borders
#     #ax.add_feature(cfeature.LAND, facecolor='0.8')
#     ax.coastlines(ls = '-', linewidth = 5, color = 'k')

#     #plot MIMIC TPW
#     tpw_levels = np.arange(0, 70.5, 1)
#     pm0 = ax.contourf(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, levels = tpw_levels,
#                       extend = 'max', cmap = cm.jet, transform = data_proj, zorder = 0)
# #             pm0 = ax.pcolormesh(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, vmin = 0, vmax = 70,
# #                                 cmap = cm.jet, transform = data_proj, zorder = 0)

# #     #plot ERA5 RH
# #     tpw_levels = np.arange(0, 100.1, 2)  #actually RH levels, but keeping the tpw_levels name because we use it elsewhere
# #     pm0 = ax.contourf(ds.longitude, ds.latitude, rh[0].values, levels = tpw_levels,
# #                       extend = 'max', cmap = cm.jet, transform = data_proj, alpha = 0.6, zorder = 1)
# # #             pm0 = ax.pcolormesh(ds.longitude, ds.latitude, rh[0].values, vmin = 0, vmax = 70,
# # #                                 cmap = cm.jet, transform = data_proj, zorder = 1)

#     #plot IMERG Rain Rate
#     pm1 = ax.contourf(imerg_lons, imerg_lats, imerg_precip_masked, 
#                       levels = np.logspace(np.log10(0.1), np.log10(40), num = len(tpw_levels)), 
#                       norm = 'log', extend = 'max', 
#                       cmap = cm.jet, transform = data_proj, zorder = 1)
# #             pm1 = ax.pcolormesh(imerg_lons, imerg_lats, imerg_precip_masked, 
# #                                 norm = mplc.LogNorm(vmin = 0.1, vmax = 40), 
# #                                 cmap = cm.jet, transform = data_proj, zorder = 1)

#     #gridlines
#     gl = ax.gridlines(crs = ccrs.PlateCarree(), draw_labels = True, linewidth = 2, 
#                       color = 'gray', alpha = 0.5, linestyle = '--', zorder = 3)
#     gl.top_labels = False
#     gl.right_labels = False
#     gl.xlabel_style = {'size':45, 'color':'black'}
#     gl.ylabel_style = {'size':45, 'color':'black'}

#     # #plot ERA5 streamlines
#     # ax.streamplot(ds.longitude, ds.latitude, uwnd[0].values, vwnd[0].values,
#     #               color = 'white', linewidth = 4, density = 1.0, 
#     #               arrowsize = 6, arrowstyle = '->', 
#     #               transform = data_proj, zorder = 3)  

#     #plot flight track
#     ax.plot(flight_lon, flight_lat, color = 'darkmagenta', linewidth = 5, zorder = 4)

# #     #plot dropsonde locations for the given flight
# #     for sonde in drop_coords_and_time:
# #         #ax.scatter(sonde[0], sonde[1], marker = f'${sonde[2]}$', color = 'b', s = 300)
# #         if sonde[2] in ['15:10', '15:43', '16:16', '16:42']:
# #             ax.scatter(sonde[0], sonde[1], marker = '*', color = 'white', edgecolor = 'k', zorder = 5, s = 6000)
# #         else:
# #             ax.scatter(sonde[0], sonde[1], marker = '*', color = 'k', zorder = 5, s = 6000)

# #     #same as above, but labeling the dropsondes by the order that they appear in the
# #         #Dropsonde_Metric_Calculations.csv, NOT IN CHRONOLOGICAL ORDER!!!
# #     #for z, sonde in enumerate(drop_coords_and_time):
# #         #ax.scatter(sonde[0], sonde[1], marker = f'${z + 1}$', color = 'b', s = 120, zorder = 5)

#     #plotting the colorbars
#     #cbar0 = group_fig.colorbar(pm0, ax = ax, orientation = 'vertical', shrink = 0.75, pad = 0.25)
#     #cbar0.set_label('TPW [mm]')
#     #cbar0.ax.yaxis.set_ticks_position('left')
#     #cbar0.ax.yaxis.set_label_position('left')

#     #this works with GeoAxes (i.e., Cartopy's map projections)
#     if key == len(case_dict_agu) - 1:

#         #MIMIC TPW colorbar
#         ticks_tpw = np.arange(0, 70.5, 10, dtype = int)
#         #cax = group_fig.add_axes([ax.get_position().x1+0.05, ax.get_position().y0, 0.02, ax.get_position().height])
#         cax0 = group_fig.add_axes([ax.get_position().x1 + 0.10, ax.get_position().y0, 0.04, 0.77])
#         cbar0 = group_fig.colorbar(pm0, cax = cax0, ticks = ticks_tpw)
#         cbar0.set_label('TPW [mm]')
#         cbar0.ax.set_yticklabels(list(map(str, list(ticks_tpw))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
#         cbar0.ax.yaxis.set_ticks_position('left')
#         cbar0.ax.yaxis.set_label_position('left')

#         # #ERA5 RH colorbar
#         # ticks_rh = np.arange(0, 100.5, 10, dtype = int)
#         # #cax = group_fig.add_axes([ax.get_position().x1+0.05, ax.get_position().y0, 0.02, ax.get_position().height])
#         # cax0 = group_fig.add_axes([ax.get_position().x1 + 0.10, ax.get_position().y0, 0.04, 0.77])
#         # cbar0 = group_fig.colorbar(pm0, cax = cax0, ticks = ticks_rh)
#         # cbar0.set_label('Relative Humidity [%]')
#         # cbar0.ax.set_yticklabels(list(map(str, list(ticks_rh))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
#         # cbar0.ax.yaxis.set_ticks_position('left')
#         # cbar0.ax.yaxis.set_label_position('left')

#         #IMERG colorbar
#         ticks_imerg = np.array([0.1, 1, 5, 10, 20, 40], dtype = float)
#         cax1 = group_fig.add_axes([ax.get_position().x1 + 0.10, ax.get_position().y0, 0.04, 0.77])
#         cbar1 = group_fig.colorbar(pm1, cax = cax1, ticks = ticks_imerg)
#         cbar1.set_label('IMERG [mm hr$\\bf{^{-1}}$]')
#         cbar1.ax.set_yticklabels(list(map(str, list(ticks_imerg))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
#         cbar1.ax.yaxis.set_ticks_position('right')
#         cbar1.ax.yaxis.set_label_position('right')
                
#     ds_tpw.close()
#     ds_imerg.close()
#     ds.close()

# #plt.tight_layout()
# #plt.subplots_adjust(hspace = 0.3)

# #save the figure

# #need to manually write out the whole filepath; for some reason, you can't use os.path.join() unless you're 
#     #saving in the script's directory or you're saving within a subdirectory of the script's directory
# plot_save_name = '/Users/ben/Desktop/Figure2.png'

# plt.savefig(plot_save_name, bbox_inches = 'tight')
# #plt.show()  #plt.show() must come after plt.savefig() in order for the image to save properly
# #plt.clf()   #supposedly speeds things up? According to: https://www.youtube.com/watch?v=jGVIZbi9uMY
# plt.close()

# ##decrease file size of the image by 66% without noticeable image effects (if using Matplotlib)
# ##(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
# im = Image.open(plot_save_name)

# try:
#     im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
# except:
#     #use this for older version of PIL/Pillow if the above line doesn't work, 
#     #though this line will have isolated, extremely minor image effects due to 
#     #only using 256 colors instead of the 3-element RGB scale
#     im2 = im.convert('P')

# im2.save(plot_save_name)
# im.close()
# im2.close()
